# 10 MSPS

## Training the model with 19 Features


In [ ]:
import numpy as np
import os
import joblib
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import skew, kurtosis
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score, classification_report, confusion_matrix,
    roc_curve, auc, precision_recall_curve, matthews_corrcoef, 
    balanced_accuracy_score, log_loss, f1_score, recall_score, precision_score
)

# ==================================================================
# Fixed Configuration
# ==================================================================
base_directory = "C:\\Users\\awm21\\Documents\\Probe_Vari\\Neural_Networks\\Features_Vitis"

adc_num = 1
train_probe_dataset = 1
test_probe_dataset = 2
sample_rate = 10

train_probe = f"Probe{train_probe_dataset}"
test_probe = f"Probe{test_probe_dataset}"

# List of all 38 loaded features
all_feature_names = [
    "Max_H", "Min_H", "Mean_H", "Std_H", "Mean Deviation_H", "RMS_H", "Skewness_H", "Kurtosis_H", "Peak-to-Peak_H", "Zero Crossing Rate_H",
    "Centroid_H", "Entropy_H", "Spread_H", "Skewness_Freq_H", "Mean_Freq_H", "Kurtosis_Freq_H", "Irregularity_H", "Variance_H", "Dominant_Freq_H",
    "Max_L", "Min_L", "Mean_L", "Std_L", "Mean Deviation_L", "RMS_L", "Skewness_L", "Kurtosis_L", "Peak-to-Peak_L", "Zero Crossing Rate_L",
    "Centroid_L", "Entropy_L", "Spread_L", "Skewness_Freq_L", "Mean_Freq_L", "Kurtosis_Freq_L", "Irregularity_L", "Variance_L"
]

# Select only the 19 required features from the 38 loaded features
selected_features = [
    "Mean_H", "RMS_H", "Mean_L", "Variance_L", "RMS_L", "Mean_Freq_H", "Entropy_L", 
    "Max_H", "Entropy_H", "Mean Deviation_H", "Min_L", "Variance_H", "Mean_Freq_L", 
    "Min_H", "Mean Deviation_L", "Max_L", "Std_L", "Skewness_H", "Skewness_L"
]

# Get indices of selected features in the original dataset
selected_feature_indices = [all_feature_names.index(f) for f in selected_features]

def load_channel_data(probe_name, adc_num, channel):
    """Load specific ADC/channel data for a probe and verify ADC consistency."""
    folder = os.path.join(base_directory, f"P{probe_name[-1]}_fd")
    filename = f'fd_{probe_name}_ADC{adc_num}_CH{channel}_{sample_rate}_MSPS.npy'
    file_path = os.path.join(folder, filename)
    
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"File not found: {file_path}")
    
    data = np.load(file_path)
    print(f"✅ Loaded {probe_name} ADC{adc_num} CH{channel}: {data.shape}")
    return data

# ==================================================================
# Training & Test Data
# ==================================================================
train_ch0 = load_channel_data(train_probe, adc_num, 1)
train_ch1 = load_channel_data(train_probe, adc_num, 2)
test_ch0 = load_channel_data(test_probe, adc_num, 1)
test_ch1 = load_channel_data(test_probe, adc_num, 2)

# Stack training and test data
X_train = np.vstack([train_ch0, train_ch1])
y_train = np.concatenate([np.zeros(train_ch0.shape[0]), np.ones(train_ch1.shape[0])])
X_test = np.vstack([test_ch0, test_ch1])
y_test = np.concatenate([np.zeros(test_ch0.shape[0]), np.ones(test_ch1.shape[0])])

# ==================================================================
# Data Preprocessing
# ==================================================================
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Select only required 19 features from the 38 loaded dataset
X_train_selected = X_train_scaled[:, selected_feature_indices]
X_test_selected = X_test_scaled[:, selected_feature_indices]

# ==================================================================
# Train Model with Selected Features (19 Features)
# ==================================================================
rf = RandomForestClassifier(
    n_estimators=50,
    max_depth=10,
    min_samples_split=10,
    min_samples_leaf=5,
    max_features='sqrt',
    random_state=42
)
rf.fit(X_train_selected, y_train)

# Get feature importance only for selected features
feature_importances = rf.feature_importances_

# Print Selected Features and their Importances
print("\nSelected Features and Importances:")
for feature_idx, feature_name in zip(range(len(selected_features)), selected_features):
    print(f"{feature_name}: Importance {feature_importances[feature_idx]:.4f}")

# ==================================================================
# Retrain Model with Selected Features
# ==================================================================
rf_selected = RandomForestClassifier(
    n_estimators=50,
    max_depth=10,
    min_samples_split=10,
    min_samples_leaf=5,
    max_features='sqrt',
    random_state=42
)
rf_selected.fit(X_train_selected, y_train)

# Create a unique folder name dynamically
model_dir = f"Models_test/19_Feature_Tuning_{train_probe}_to_{test_probe}_ADC{adc_num}_{sample_rate}_MSPS"
os.makedirs(model_dir, exist_ok=True)

# Save the Model and Scaler
joblib.dump(rf_selected, os.path.join(model_dir, "RandomForest_Model.pkl"))
joblib.dump(scaler, os.path.join(model_dir, "Scaler.pkl"))
joblib.dump(selected_feature_indices, os.path.join(model_dir, "Selected_Features.pkl"))

# Predictions
y_pred_selected = rf_selected.predict(X_test_selected)
y_probs_selected = rf_selected.predict_proba(X_test_selected)[:, 1]

# ==================================================================
# Performance Metrics After Feature Selection
# ==================================================================
print("\nModel Performance Using 19 Features:")
print(f"Test Accuracy: {accuracy_score(y_test, y_pred_selected):.4f}")
print(f"Balanced Accuracy: {balanced_accuracy_score(y_test, y_pred_selected):.4f}")
print(f"MCC: {matthews_corrcoef(y_test, y_pred_selected):.4f}")
print(f"Log Loss: {log_loss(y_test, y_probs_selected):.4f}")
print(f"F1 Score: {f1_score(y_test, y_pred_selected):.4f}")
print(f"Recall: {recall_score(y_test, y_pred_selected):.4f}")
print(f"Precision: {precision_score(y_test, y_pred_selected):.4f}")
print(classification_report(y_test, y_pred_selected, target_names=['CH1', 'CH2']))

# Compute Confusion Matrix, FPR, and FNR
cm = confusion_matrix(y_test, y_pred_selected)
print("\nConfusion Matrix:")
print(cm)
fp_rate = cm[0][1] / (cm[0][1] + cm[0][0])
fn_rate = cm[1][0] / (cm[1][0] + cm[1][1])
print(f"False Positive Rate (FPR): {fp_rate:.4f}")
print(f"False Negative Rate (FNR): {fn_rate:.4f}")


✅ Loaded Probe1 ADC1 CH1: (9998, 38)
✅ Loaded Probe1 ADC1 CH2: (9998, 38)
✅ Loaded Probe2 ADC1 CH1: (9998, 38)
✅ Loaded Probe2 ADC1 CH2: (9998, 38)

Selected Features and Importances:
Mean_H: Importance 0.2014
RMS_H: Importance 0.1676
Mean_L: Importance 0.0825
Variance_L: Importance 0.0524
RMS_L: Importance 0.0807
Mean_Freq_H: Importance 0.0285
Entropy_L: Importance 0.0177
Max_H: Importance 0.0459
Entropy_H: Importance 0.0973
Mean Deviation_H: Importance 0.0128
Min_L: Importance 0.0351
Variance_H: Importance 0.0903
Mean_Freq_L: Importance 0.0061
Min_H: Importance 0.0052
Mean Deviation_L: Importance 0.0139
Max_L: Importance 0.0356
Std_L: Importance 0.0130
Skewness_H: Importance 0.0047
Skewness_L: Importance 0.0092

Model Performance Using 19 Features:
Test Accuracy: 0.9574
Balanced Accuracy: 0.9574
MCC: 0.9169
Log Loss: 0.1001
F1 Score: 0.9588
Recall: 0.9915
Precision: 0.9282
              precision    recall  f1-score   support

         CH1       0.99      0.92      0.96      9998
   

## Testing on the models trained from above

### ADC2 Probe3 Test at 10MSPS

In [ ]:
import numpy as np
import os
import joblib
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score, classification_report, confusion_matrix, log_loss,
    f1_score, recall_score, precision_score, matthews_corrcoef, balanced_accuracy_score
)

# ==================================================================
# Fixed Configuration
# ==================================================================
base_directory = "C:\\Users\\awm21\\Documents\\Probe_Vari\\Neural_Networks\\Features_Vitis"

adc_num = 2
test_probe_dataset = 3
sample_rate = 10

test_probe = f"Probe{test_probe_dataset}"

# Directory where the model and scaler were saved
model_dir = f"Models_test/19_Feature_Tuning_Probe1_to_Probe2_ADC1_10_MSPS"

# Load the trained model and scaler
model_path = os.path.join(model_dir, "RandomForest_Model.pkl")
scaler_path = os.path.join(model_dir, "Scaler.pkl")
features_path = os.path.join(model_dir, "Selected_Features.pkl")

if not os.path.exists(model_path) or not os.path.exists(scaler_path) or not os.path.exists(features_path):
    raise FileNotFoundError("Model, scaler, or selected features file not found.")

rf_selected = joblib.load(model_path)
scaler = joblib.load(scaler_path)
selected_feature_indices = joblib.load(features_path)

# List of all 38 loaded features
all_feature_names = [
    "Max_H", "Min_H", "Mean_H", "Std_H", "Mean Deviation_H", "RMS_H", "Skewness_H", "Kurtosis_H", "Peak-to-Peak_H", "Zero Crossing Rate_H",
    "Centroid_H", "Entropy_H", "Spread_H", "Skewness_Freq_H", "Mean_Freq_H", "Kurtosis_Freq_H", "Irregularity_H", "Variance_H", "Dominant_Freq_H",
    "Max_L", "Min_L", "Mean_L", "Std_L", "Mean Deviation_L", "RMS_L", "Skewness_L", "Kurtosis_L", "Peak-to-Peak_L", "Zero Crossing Rate_L",
    "Centroid_L", "Entropy_L", "Spread_L", "Skewness_Freq_L", "Mean_Freq_L", "Kurtosis_Freq_L", "Irregularity_L", "Variance_L"
]

# Get feature names for selected features
selected_features = [all_feature_names[i] for i in selected_feature_indices]

def load_channel_data(probe_name, adc_num, channel):
    """Load specific ADC/channel data for a probe and verify ADC consistency."""
    folder = os.path.join(base_directory, f"P{probe_name[-1]}_fd")
    filename = f'fd_{probe_name}_ADC{adc_num}_CH{channel}_{sample_rate}_MSPS.npy'
    file_path = os.path.join(folder, filename)
    
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"File not found: {file_path}")
    
    data = np.load(file_path)
    print(f"✅ Loaded {probe_name} ADC{adc_num} CH{channel}: {data.shape}")
    return data

# ==================================================================
# Load Test Data
# ==================================================================
test_ch0 = load_channel_data(test_probe, adc_num, 1)
test_ch1 = load_channel_data(test_probe, adc_num, 2)

# Stack test data
X_test = np.vstack([test_ch0, test_ch1])
y_test = np.concatenate([np.zeros(test_ch0.shape[0]), np.ones(test_ch1.shape[0])])

# Apply standardization using the loaded scaler
X_test_scaled = scaler.transform(X_test)

# Select only required 19 features from the 38 loaded dataset
X_test_selected = X_test_scaled[:, selected_feature_indices]

# ==================================================================
# Evaluate the Model
# ==================================================================
y_pred_selected = rf_selected.predict(X_test_selected)
y_probs_selected = rf_selected.predict_proba(X_test_selected)[:, 1]

# Print Model Performance
print("\nModel Performance Using 19 Features:")
print(f"Test Accuracy: {accuracy_score(y_test, y_pred_selected):.4f}")
print(f"Balanced Accuracy: {balanced_accuracy_score(y_test, y_pred_selected):.4f}")
print(f"MCC: {matthews_corrcoef(y_test, y_pred_selected):.4f}")
print(f"Log Loss: {log_loss(y_test, y_probs_selected):.4f}")
print(f"F1 Score: {f1_score(y_test, y_pred_selected):.4f}")
print(f"Recall: {recall_score(y_test, y_pred_selected):.4f}")
print(f"Precision: {precision_score(y_test, y_pred_selected):.4f}")
print(classification_report(y_test, y_pred_selected, target_names=['CH1', 'CH2']))

# Compute Confusion Matrix, FPR, and FNR
cm = confusion_matrix(y_test, y_pred_selected)
print("\nConfusion Matrix:")
print(cm)
fp_rate = cm[0][1] / (cm[0][1] + cm[0][0])
fn_rate = cm[1][0] / (cm[1][0] + cm[1][1])
print(f"False Positive Rate (FPR): {fp_rate:.4f}")
print(f"False Negative Rate (FNR): {fn_rate:.4f}")


✅ Loaded Probe3 ADC2 CH1: (9998, 38)
✅ Loaded Probe3 ADC2 CH2: (9998, 38)

Model Performance Using 19 Features:
Test Accuracy: 0.9520
Balanced Accuracy: 0.9520
MCC: 0.9082
Log Loss: 0.1638
F1 Score: 0.9497
Recall: 0.9047
Precision: 0.9993
              precision    recall  f1-score   support

         CH1       0.91      1.00      0.95      9998
         CH2       1.00      0.90      0.95      9998

    accuracy                           0.95     19996
   macro avg       0.96      0.95      0.95     19996
weighted avg       0.96      0.95      0.95     19996


Confusion Matrix:
[[9992    6]
 [ 953 9045]]
False Positive Rate (FPR): 0.0006
False Negative Rate (FNR): 0.0953


### ADC3 Probe3 Test at 10MSPS

In [5]:
import numpy as np
import os
import joblib
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score, classification_report, confusion_matrix, log_loss,
    f1_score, recall_score, precision_score, matthews_corrcoef, balanced_accuracy_score
)

# ==================================================================
# Fixed Configuration
# ==================================================================
base_directory = "C:\\Users\\awm21\\Documents\\Probe_Vari\\Neural_Networks\\Features_Vitis"

adc_num = 3
test_probe_dataset = 3
sample_rate = 10

test_probe = f"Probe{test_probe_dataset}"

# Directory where the model and scaler were saved
model_dir = f"Models_test/19_Feature_Tuning_Probe1_to_Probe2_ADC1_10_MSPS"

# Load the trained model and scaler
model_path = os.path.join(model_dir, "RandomForest_Model.pkl")
scaler_path = os.path.join(model_dir, "Scaler.pkl")
features_path = os.path.join(model_dir, "Selected_Features.pkl")

if not os.path.exists(model_path) or not os.path.exists(scaler_path) or not os.path.exists(features_path):
    raise FileNotFoundError("Model, scaler, or selected features file not found.")

rf_selected = joblib.load(model_path)
scaler = joblib.load(scaler_path)
selected_feature_indices = joblib.load(features_path)

# List of all 38 loaded features
all_feature_names = [
    "Max_H", "Min_H", "Mean_H", "Std_H", "Mean Deviation_H", "RMS_H", "Skewness_H", "Kurtosis_H", "Peak-to-Peak_H", "Zero Crossing Rate_H",
    "Centroid_H", "Entropy_H", "Spread_H", "Skewness_Freq_H", "Mean_Freq_H", "Kurtosis_Freq_H", "Irregularity_H", "Variance_H", "Dominant_Freq_H",
    "Max_L", "Min_L", "Mean_L", "Std_L", "Mean Deviation_L", "RMS_L", "Skewness_L", "Kurtosis_L", "Peak-to-Peak_L", "Zero Crossing Rate_L",
    "Centroid_L", "Entropy_L", "Spread_L", "Skewness_Freq_L", "Mean_Freq_L", "Kurtosis_Freq_L", "Irregularity_L", "Variance_L"
]

# Get feature names for selected features
selected_features = [all_feature_names[i] for i in selected_feature_indices]

def load_channel_data(probe_name, adc_num, channel):
    """Load specific ADC/channel data for a probe and verify ADC consistency."""
    folder = os.path.join(base_directory, f"P{probe_name[-1]}_fd")
    filename = f'fd_{probe_name}_ADC{adc_num}_CH{channel}_{sample_rate}_MSPS.npy'
    file_path = os.path.join(folder, filename)
    
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"File not found: {file_path}")
    
    data = np.load(file_path)
    print(f"✅ Loaded {probe_name} ADC{adc_num} CH{channel}: {data.shape}")
    return data

# ==================================================================
# Load Test Data
# ==================================================================
test_ch0 = load_channel_data(test_probe, adc_num, 1)
test_ch1 = load_channel_data(test_probe, adc_num, 2)

# Stack test data
X_test = np.vstack([test_ch0, test_ch1])
y_test = np.concatenate([np.zeros(test_ch0.shape[0]), np.ones(test_ch1.shape[0])])

# Apply standardization using the loaded scaler
X_test_scaled = scaler.transform(X_test)

# Select only required 19 features from the 38 loaded dataset
X_test_selected = X_test_scaled[:, selected_feature_indices]

# ==================================================================
# Evaluate the Model
# ==================================================================
y_pred_selected = rf_selected.predict(X_test_selected)
y_probs_selected = rf_selected.predict_proba(X_test_selected)[:, 1]

# Print Model Performance
print("\nModel Performance Using 19 Features:")
print(f"Test Accuracy: {accuracy_score(y_test, y_pred_selected):.4f}")
print(f"Balanced Accuracy: {balanced_accuracy_score(y_test, y_pred_selected):.4f}")
print(f"MCC: {matthews_corrcoef(y_test, y_pred_selected):.4f}")
print(f"Log Loss: {log_loss(y_test, y_probs_selected):.4f}")
print(f"F1 Score: {f1_score(y_test, y_pred_selected):.4f}")
print(f"Recall: {recall_score(y_test, y_pred_selected):.4f}")
print(f"Precision: {precision_score(y_test, y_pred_selected):.4f}")
print(classification_report(y_test, y_pred_selected, target_names=['CH1', 'CH2']))

# Compute Confusion Matrix, FPR, and FNR
cm = confusion_matrix(y_test, y_pred_selected)
print("\nConfusion Matrix:")
print(cm)
fp_rate = cm[0][1] / (cm[0][1] + cm[0][0])
fn_rate = cm[1][0] / (cm[1][0] + cm[1][1])
print(f"False Positive Rate (FPR): {fp_rate:.4f}")
print(f"False Negative Rate (FNR): {fn_rate:.4f}")


✅ Loaded Probe3 ADC3 CH1: (9998, 38)
✅ Loaded Probe3 ADC3 CH2: (9998, 38)

Model Performance Using 19 Features:
Test Accuracy: 0.9985
Balanced Accuracy: 0.9985
MCC: 0.9971
Log Loss: 0.0092
F1 Score: 0.9985
Recall: 0.9971
Precision: 1.0000
              precision    recall  f1-score   support

         CH1       1.00      1.00      1.00      9998
         CH2       1.00      1.00      1.00      9998

    accuracy                           1.00     19996
   macro avg       1.00      1.00      1.00     19996
weighted avg       1.00      1.00      1.00     19996


Confusion Matrix:
[[9998    0]
 [  29 9969]]
False Positive Rate (FPR): 0.0000
False Negative Rate (FNR): 0.0029


# 20 MSPS

## Training the model with 19 Features


In [4]:
import numpy as np
import os
import joblib
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import skew, kurtosis
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score, classification_report, confusion_matrix,
    roc_curve, auc, precision_recall_curve, matthews_corrcoef, 
    balanced_accuracy_score, log_loss, f1_score, recall_score, precision_score
)

# ==================================================================
# Fixed Configuration
# ==================================================================
base_directory = "C:\\Users\\awm21\\Documents\\Probe_Vari\\Neural_Networks\\Features_Vitis"

adc_num = 1
train_probe_dataset = 1
test_probe_dataset = 2
sample_rate = 20

train_probe = f"Probe{train_probe_dataset}"
test_probe = f"Probe{test_probe_dataset}"

# List of all 38 loaded features
all_feature_names = [
    "Max_H", "Min_H", "Mean_H", "Std_H", "Mean Deviation_H", "RMS_H", "Skewness_H", "Kurtosis_H", "Peak-to-Peak_H", "Zero Crossing Rate_H",
    "Centroid_H", "Entropy_H", "Spread_H", "Skewness_Freq_H", "Mean_Freq_H", "Kurtosis_Freq_H", "Irregularity_H", "Variance_H", "Dominant_Freq_H",
    "Max_L", "Min_L", "Mean_L", "Std_L", "Mean Deviation_L", "RMS_L", "Skewness_L", "Kurtosis_L", "Peak-to-Peak_L", "Zero Crossing Rate_L",
    "Centroid_L", "Entropy_L", "Spread_L", "Skewness_Freq_L", "Mean_Freq_L", "Kurtosis_Freq_L", "Irregularity_L", "Variance_L"
]

# Select only the 19 required features from the 38 loaded features
selected_features = [
    "Mean_H", "RMS_H", "Mean_L", "Variance_L", "RMS_L", "Mean_Freq_H", "Entropy_L", 
    "Max_H", "Entropy_H", "Mean Deviation_H", "Min_L", "Variance_H", "Mean_Freq_L", 
    "Min_H", "Mean Deviation_L", "Max_L", "Std_L", "Skewness_H", "Skewness_L"
]

# Get indices of selected features in the original dataset
selected_feature_indices = [all_feature_names.index(f) for f in selected_features]

def load_channel_data(probe_name, adc_num, channel):
    """Load specific ADC/channel data for a probe and verify ADC consistency."""
    folder = os.path.join(base_directory, f"P{probe_name[-1]}_fd")
    filename = f'fd_{probe_name}_ADC{adc_num}_CH{channel}_{sample_rate}_MSPS.npy'
    file_path = os.path.join(folder, filename)
    
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"File not found: {file_path}")
    
    data = np.load(file_path)
    print(f"✅ Loaded {probe_name} ADC{adc_num} CH{channel}: {data.shape}")
    return data

# ==================================================================
# Training & Test Data
# ==================================================================
train_ch0 = load_channel_data(train_probe, adc_num, 1)
train_ch1 = load_channel_data(train_probe, adc_num, 2)
test_ch0 = load_channel_data(test_probe, adc_num, 1)
test_ch1 = load_channel_data(test_probe, adc_num, 2)

# Stack training and test data
X_train = np.vstack([train_ch0, train_ch1])
y_train = np.concatenate([np.zeros(train_ch0.shape[0]), np.ones(train_ch1.shape[0])])
X_test = np.vstack([test_ch0, test_ch1])
y_test = np.concatenate([np.zeros(test_ch0.shape[0]), np.ones(test_ch1.shape[0])])

# ==================================================================
# Data Preprocessing
# ==================================================================
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Select only required 19 features from the 38 loaded dataset
X_train_selected = X_train_scaled[:, selected_feature_indices]
X_test_selected = X_test_scaled[:, selected_feature_indices]

# ==================================================================
# Train Model with Selected Features (19 Features)
# ==================================================================
rf = RandomForestClassifier(
    n_estimators=50,
    max_depth=10,
    min_samples_split=10,
    min_samples_leaf=5,
    max_features='sqrt',
    random_state=42
)
rf.fit(X_train_selected, y_train)

# Get feature importance only for selected features
feature_importances = rf.feature_importances_

# Print Selected Features and their Importances
print("\nSelected Features and Importances:")
for feature_idx, feature_name in zip(range(len(selected_features)), selected_features):
    print(f"{feature_name}: Importance {feature_importances[feature_idx]:.4f}")

# ==================================================================
# Retrain Model with Selected Features
# ==================================================================
rf_selected = RandomForestClassifier(
    n_estimators=50,
    max_depth=10,
    min_samples_split=10,
    min_samples_leaf=5,
    max_features='sqrt',
    random_state=42
)
rf_selected.fit(X_train_selected, y_train)

# Create a unique folder name dynamically
model_dir = f"Models_test/19_Feature_Tuning_{train_probe}_to_{test_probe}_ADC{adc_num}_{sample_rate}_MSPS"
os.makedirs(model_dir, exist_ok=True)

# Save the Model and Scaler
joblib.dump(rf_selected, os.path.join(model_dir, "RandomForest_Model.pkl"))
joblib.dump(scaler, os.path.join(model_dir, "Scaler.pkl"))
joblib.dump(selected_feature_indices, os.path.join(model_dir, "Selected_Features.pkl"))

# Predictions
y_pred_selected = rf_selected.predict(X_test_selected)
y_probs_selected = rf_selected.predict_proba(X_test_selected)[:, 1]

# ==================================================================
# Performance Metrics After Feature Selection
# ==================================================================
print("\nModel Performance Using 19 Features:")
print(f"Test Accuracy: {accuracy_score(y_test, y_pred_selected):.4f}")
print(f"Balanced Accuracy: {balanced_accuracy_score(y_test, y_pred_selected):.4f}")
print(f"MCC: {matthews_corrcoef(y_test, y_pred_selected):.4f}")
print(f"Log Loss: {log_loss(y_test, y_probs_selected):.4f}")
print(f"F1 Score: {f1_score(y_test, y_pred_selected):.4f}")
print(f"Recall: {recall_score(y_test, y_pred_selected):.4f}")
print(f"Precision: {precision_score(y_test, y_pred_selected):.4f}")
print(classification_report(y_test, y_pred_selected, target_names=['CH1', 'CH2']))

# Compute Confusion Matrix, FPR, and FNR
cm = confusion_matrix(y_test, y_pred_selected)
print("\nConfusion Matrix:")
print(cm)
fp_rate = cm[0][1] / (cm[0][1] + cm[0][0])
fn_rate = cm[1][0] / (cm[1][0] + cm[1][1])
print(f"False Positive Rate (FPR): {fp_rate:.4f}")
print(f"False Negative Rate (FNR): {fn_rate:.4f}")


✅ Loaded Probe1 ADC1 CH1: (4998, 38)
✅ Loaded Probe1 ADC1 CH2: (4998, 38)
✅ Loaded Probe2 ADC1 CH1: (4998, 38)
✅ Loaded Probe2 ADC1 CH2: (4998, 38)

Selected Features and Importances:
Mean_H: Importance 0.2719
RMS_H: Importance 0.1919
Mean_L: Importance 0.1784
Variance_L: Importance 0.0175
RMS_L: Importance 0.1202
Mean_Freq_H: Importance 0.0316
Entropy_L: Importance 0.0014
Max_H: Importance 0.0406
Entropy_H: Importance 0.0840
Mean Deviation_H: Importance 0.0000
Min_L: Importance 0.0028
Variance_H: Importance 0.0413
Mean_Freq_L: Importance 0.0025
Min_H: Importance 0.0135
Mean Deviation_L: Importance 0.0002
Max_L: Importance 0.0000
Std_L: Importance 0.0019
Skewness_H: Importance 0.0001
Skewness_L: Importance 0.0000

Model Performance Using 19 Features:
Test Accuracy: 0.9999
Balanced Accuracy: 0.9999
MCC: 0.9998
Log Loss: 0.0020
F1 Score: 0.9999
Recall: 0.9998
Precision: 1.0000
              precision    recall  f1-score   support

         CH1       1.00      1.00      1.00      4998
   

## Testing on the models trained from above

### ADC2 Probe 3 Test at 20 MSPS


In [39]:
import numpy as np
import os
import joblib
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score, classification_report, confusion_matrix, log_loss,
    f1_score, recall_score, precision_score, matthews_corrcoef, balanced_accuracy_score
)

# ==================================================================
# Fixed Configuration
# ==================================================================
base_directory = "C:\\Users\\awm21\\Documents\\Probe_Vari\\Neural_Networks\\Features_Vitis"

adc_num = 2
test_probe_dataset = 3
sample_rate = 20

test_probe = f"Probe{test_probe_dataset}"

# Directory where the model and scaler were saved
model_dir = f"Models_test/19_Feature_Tuning_Probe1_to_Probe2_ADC1_20_MSPS"

# Load the trained model and scaler
model_path = os.path.join(model_dir, "RandomForest_Model.pkl")
scaler_path = os.path.join(model_dir, "Scaler.pkl")
features_path = os.path.join(model_dir, "Selected_Features.pkl")

if not os.path.exists(model_path) or not os.path.exists(scaler_path) or not os.path.exists(features_path):
    raise FileNotFoundError("Model, scaler, or selected features file not found.")

rf_selected = joblib.load(model_path)
scaler = joblib.load(scaler_path)
selected_feature_indices = joblib.load(features_path)

# List of all 38 loaded features
all_feature_names = [
    "Max_H", "Min_H", "Mean_H", "Std_H", "Mean Deviation_H", "RMS_H", "Skewness_H", "Kurtosis_H", "Peak-to-Peak_H", "Zero Crossing Rate_H",
    "Centroid_H", "Entropy_H", "Spread_H", "Skewness_Freq_H", "Mean_Freq_H", "Kurtosis_Freq_H", "Irregularity_H", "Variance_H", "Dominant_Freq_H",
    "Max_L", "Min_L", "Mean_L", "Std_L", "Mean Deviation_L", "RMS_L", "Skewness_L", "Kurtosis_L", "Peak-to-Peak_L", "Zero Crossing Rate_L",
    "Centroid_L", "Entropy_L", "Spread_L", "Skewness_Freq_L", "Mean_Freq_L", "Kurtosis_Freq_L", "Irregularity_L", "Variance_L"
]

# Get feature names for selected features
selected_features = [all_feature_names[i] for i in selected_feature_indices]

def load_channel_data(probe_name, adc_num, channel):
    """Load specific ADC/channel data for a probe and verify ADC consistency."""
    folder = os.path.join(base_directory, f"P{probe_name[-1]}_fd")
    filename = f'fd_{probe_name}_ADC{adc_num}_CH{channel}_{sample_rate}_MSPS.npy'
    file_path = os.path.join(folder, filename)
    
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"File not found: {file_path}")
    
    data = np.load(file_path)
    print(f"✅ Loaded {probe_name} ADC{adc_num} CH{channel}: {data.shape}")
    return data

# ==================================================================
# Load Test Data
# ==================================================================
test_ch0 = load_channel_data(test_probe, adc_num, 1)
test_ch1 = load_channel_data(test_probe, adc_num, 2)

# Stack test data
X_test = np.vstack([test_ch0, test_ch1])
y_test = np.concatenate([np.zeros(test_ch0.shape[0]), np.ones(test_ch1.shape[0])])

# Apply standardization using the loaded scaler
X_test_scaled = scaler.transform(X_test)

# Select only required 19 features from the 38 loaded dataset
X_test_selected = X_test_scaled[:, selected_feature_indices]

# ==================================================================
# Evaluate the Model
# ==================================================================
y_pred_selected = rf_selected.predict(X_test_selected)
y_probs_selected = rf_selected.predict_proba(X_test_selected)[:, 1]

# Print Model Performance
print("\nModel Performance Using 19 Features:")
print(f"Test Accuracy: {accuracy_score(y_test, y_pred_selected):.4f}")
print(f"Balanced Accuracy: {balanced_accuracy_score(y_test, y_pred_selected):.4f}")
print(f"MCC: {matthews_corrcoef(y_test, y_pred_selected):.4f}")
print(f"Log Loss: {log_loss(y_test, y_probs_selected):.4f}")
print(f"F1 Score: {f1_score(y_test, y_pred_selected):.4f}")
print(f"Recall: {recall_score(y_test, y_pred_selected):.4f}")
print(f"Precision: {precision_score(y_test, y_pred_selected):.4f}")
print(classification_report(y_test, y_pred_selected, target_names=['CH1', 'CH2']))

# Compute Confusion Matrix, FPR, and FNR
cm = confusion_matrix(y_test, y_pred_selected)
print("\nConfusion Matrix:")
print(cm)
fp_rate = cm[0][1] / (cm[0][1] + cm[0][0])
fn_rate = cm[1][0] / (cm[1][0] + cm[1][1])
print(f"False Positive Rate (FPR): {fp_rate:.4f}")
print(f"False Negative Rate (FNR): {fn_rate:.4f}")


✅ Loaded Probe3 ADC2 CH1: (4998, 38)
✅ Loaded Probe3 ADC2 CH2: (4998, 38)

Model Performance Using 19 Features:
Test Accuracy: 1.0000
Balanced Accuracy: 1.0000
MCC: 1.0000
Log Loss: 0.0003
F1 Score: 1.0000
Recall: 1.0000
Precision: 1.0000
              precision    recall  f1-score   support

         CH1       1.00      1.00      1.00      4998
         CH2       1.00      1.00      1.00      4998

    accuracy                           1.00      9996
   macro avg       1.00      1.00      1.00      9996
weighted avg       1.00      1.00      1.00      9996


Confusion Matrix:
[[4998    0]
 [   0 4998]]
False Positive Rate (FPR): 0.0000
False Negative Rate (FNR): 0.0000


### ADC3 Probe3 Test at 20MSPS

In [40]:
import numpy as np
import os
import joblib
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score, classification_report, confusion_matrix, log_loss,
    f1_score, recall_score, precision_score, matthews_corrcoef, balanced_accuracy_score
)

# ==================================================================
# Fixed Configuration
# ==================================================================
base_directory = "C:\\Users\\awm21\\Documents\\Probe_Vari\\Neural_Networks\\Features_Vitis"

adc_num = 3
test_probe_dataset = 3
sample_rate = 20

test_probe = f"Probe{test_probe_dataset}"

# Directory where the model and scaler were saved
model_dir = f"Models_test/19_Feature_Tuning_Probe1_to_Probe2_ADC1_20_MSPS"

# Load the trained model and scaler
model_path = os.path.join(model_dir, "RandomForest_Model.pkl")
scaler_path = os.path.join(model_dir, "Scaler.pkl")
features_path = os.path.join(model_dir, "Selected_Features.pkl")

if not os.path.exists(model_path) or not os.path.exists(scaler_path) or not os.path.exists(features_path):
    raise FileNotFoundError("Model, scaler, or selected features file not found.")

rf_selected = joblib.load(model_path)
scaler = joblib.load(scaler_path)
selected_feature_indices = joblib.load(features_path)

# List of all 38 loaded features
all_feature_names = [
    "Max_H", "Min_H", "Mean_H", "Std_H", "Mean Deviation_H", "RMS_H", "Skewness_H", "Kurtosis_H", "Peak-to-Peak_H", "Zero Crossing Rate_H",
    "Centroid_H", "Entropy_H", "Spread_H", "Skewness_Freq_H", "Mean_Freq_H", "Kurtosis_Freq_H", "Irregularity_H", "Variance_H", "Dominant_Freq_H",
    "Max_L", "Min_L", "Mean_L", "Std_L", "Mean Deviation_L", "RMS_L", "Skewness_L", "Kurtosis_L", "Peak-to-Peak_L", "Zero Crossing Rate_L",
    "Centroid_L", "Entropy_L", "Spread_L", "Skewness_Freq_L", "Mean_Freq_L", "Kurtosis_Freq_L", "Irregularity_L", "Variance_L"
]

# Get feature names for selected features
selected_features = [all_feature_names[i] for i in selected_feature_indices]

def load_channel_data(probe_name, adc_num, channel):
    """Load specific ADC/channel data for a probe and verify ADC consistency."""
    folder = os.path.join(base_directory, f"P{probe_name[-1]}_fd")
    filename = f'fd_{probe_name}_ADC{adc_num}_CH{channel}_{sample_rate}_MSPS.npy'
    file_path = os.path.join(folder, filename)
    
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"File not found: {file_path}")
    
    data = np.load(file_path)
    print(f"✅ Loaded {probe_name} ADC{adc_num} CH{channel}: {data.shape}")
    return data

# ==================================================================
# Load Test Data
# ==================================================================
test_ch0 = load_channel_data(test_probe, adc_num, 1)
test_ch1 = load_channel_data(test_probe, adc_num, 2)

# Stack test data
X_test = np.vstack([test_ch0, test_ch1])
y_test = np.concatenate([np.zeros(test_ch0.shape[0]), np.ones(test_ch1.shape[0])])

# Apply standardization using the loaded scaler
X_test_scaled = scaler.transform(X_test)

# Select only required 19 features from the 38 loaded dataset
X_test_selected = X_test_scaled[:, selected_feature_indices]

# ==================================================================
# Evaluate the Model
# ==================================================================
y_pred_selected = rf_selected.predict(X_test_selected)
y_probs_selected = rf_selected.predict_proba(X_test_selected)[:, 1]

# Print Model Performance
print("\nModel Performance Using 19 Features:")
print(f"Test Accuracy: {accuracy_score(y_test, y_pred_selected):.4f}")
print(f"Balanced Accuracy: {balanced_accuracy_score(y_test, y_pred_selected):.4f}")
print(f"MCC: {matthews_corrcoef(y_test, y_pred_selected):.4f}")
print(f"Log Loss: {log_loss(y_test, y_probs_selected):.4f}")
print(f"F1 Score: {f1_score(y_test, y_pred_selected):.4f}")
print(f"Recall: {recall_score(y_test, y_pred_selected):.4f}")
print(f"Precision: {precision_score(y_test, y_pred_selected):.4f}")
print(classification_report(y_test, y_pred_selected, target_names=['CH1', 'CH2']))

# Compute Confusion Matrix, FPR, and FNR
cm = confusion_matrix(y_test, y_pred_selected)
print("\nConfusion Matrix:")
print(cm)
fp_rate = cm[0][1] / (cm[0][1] + cm[0][0])
fn_rate = cm[1][0] / (cm[1][0] + cm[1][1])
print(f"False Positive Rate (FPR): {fp_rate:.4f}")
print(f"False Negative Rate (FNR): {fn_rate:.4f}")


✅ Loaded Probe3 ADC3 CH1: (4998, 38)
✅ Loaded Probe3 ADC3 CH2: (4998, 38)

Model Performance Using 19 Features:
Test Accuracy: 1.0000
Balanced Accuracy: 1.0000
MCC: 1.0000
Log Loss: 0.0011
F1 Score: 1.0000
Recall: 1.0000
Precision: 1.0000
              precision    recall  f1-score   support

         CH1       1.00      1.00      1.00      4998
         CH2       1.00      1.00      1.00      4998

    accuracy                           1.00      9996
   macro avg       1.00      1.00      1.00      9996
weighted avg       1.00      1.00      1.00      9996


Confusion Matrix:
[[4998    0]
 [   0 4998]]
False Positive Rate (FPR): 0.0000
False Negative Rate (FNR): 0.0000


# 30 MSPS

## Training the model with 19 Features

In [8]:
import numpy as np
import os
import joblib
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import skew, kurtosis
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score, classification_report, confusion_matrix,
    roc_curve, auc, precision_recall_curve, matthews_corrcoef, 
    balanced_accuracy_score, log_loss, f1_score, recall_score, precision_score
)

# ==================================================================
# Fixed Configuration
# ==================================================================
base_directory = "C:\\Users\\awm21\\Documents\\Probe_Vari\\Neural_Networks\\Features_Vitis"

adc_num = 1
train_probe_dataset = 1
test_probe_dataset = 2
sample_rate = 30

train_probe = f"Probe{train_probe_dataset}"
test_probe = f"Probe{test_probe_dataset}"

# List of all 38 loaded features
all_feature_names = [
    "Max_H", "Min_H", "Mean_H", "Std_H", "Mean Deviation_H", "RMS_H", "Skewness_H", "Kurtosis_H", "Peak-to-Peak_H", "Zero Crossing Rate_H",
    "Centroid_H", "Entropy_H", "Spread_H", "Skewness_Freq_H", "Mean_Freq_H", "Kurtosis_Freq_H", "Irregularity_H", "Variance_H", "Dominant_Freq_H",
    "Max_L", "Min_L", "Mean_L", "Std_L", "Mean Deviation_L", "RMS_L", "Skewness_L", "Kurtosis_L", "Peak-to-Peak_L", "Zero Crossing Rate_L",
    "Centroid_L", "Entropy_L", "Spread_L", "Skewness_Freq_L", "Mean_Freq_L", "Kurtosis_Freq_L", "Irregularity_L", "Variance_L"
]

# Select only the 19 required features from the 38 loaded features
selected_features = [
    "Mean_H", "RMS_H", "Mean_L", "Variance_L", "RMS_L", "Mean_Freq_H", "Entropy_L", 
    "Max_H", "Entropy_H", "Mean Deviation_H", "Min_L", "Variance_H", "Mean_Freq_L", 
    "Min_H", "Mean Deviation_L", "Max_L", "Std_L", "Skewness_H", "Skewness_L"
]

# Get indices of selected features in the original dataset
selected_feature_indices = [all_feature_names.index(f) for f in selected_features]

def load_channel_data(probe_name, adc_num, channel):
    """Load specific ADC/channel data for a probe and verify ADC consistency."""
    folder = os.path.join(base_directory, f"P{probe_name[-1]}_fd")
    filename = f'fd_{probe_name}_ADC{adc_num}_CH{channel}_{sample_rate}_MSPS.npy'
    file_path = os.path.join(folder, filename)
    
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"File not found: {file_path}")
    
    data = np.load(file_path)
    print(f"✅ Loaded {probe_name} ADC{adc_num} CH{channel}: {data.shape}")
    return data

# ==================================================================
# Training & Test Data
# ==================================================================
train_ch0 = load_channel_data(train_probe, adc_num, 1)
train_ch1 = load_channel_data(train_probe, adc_num, 2)
test_ch0 = load_channel_data(test_probe, adc_num, 1)
test_ch1 = load_channel_data(test_probe, adc_num, 2)

# Stack training and test data
X_train = np.vstack([train_ch0, train_ch1])
y_train = np.concatenate([np.zeros(train_ch0.shape[0]), np.ones(train_ch1.shape[0])])
X_test = np.vstack([test_ch0, test_ch1])
y_test = np.concatenate([np.zeros(test_ch0.shape[0]), np.ones(test_ch1.shape[0])])

# ==================================================================
# Data Preprocessing
# ==================================================================
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Select only required 19 features from the 38 loaded dataset
X_train_selected = X_train_scaled[:, selected_feature_indices]
X_test_selected = X_test_scaled[:, selected_feature_indices]

# ==================================================================
# Train Model with Selected Features (19 Features)
# ==================================================================
rf = RandomForestClassifier(
    n_estimators=50,
    max_depth=10,
    min_samples_split=10,
    min_samples_leaf=5,
    max_features='sqrt',
    random_state=42
)
rf.fit(X_train_selected, y_train)

# Get feature importance only for selected features
feature_importances = rf.feature_importances_

# Print Selected Features and their Importances
print("\nSelected Features and Importances:")
for feature_idx, feature_name in zip(range(len(selected_features)), selected_features):
    print(f"{feature_name}: Importance {feature_importances[feature_idx]:.4f}")

# ==================================================================
# Retrain Model with Selected Features
# ==================================================================
rf_selected = RandomForestClassifier(
    n_estimators=50,
    max_depth=10,
    min_samples_split=10,
    min_samples_leaf=5,
    max_features='sqrt',
    random_state=42
)
rf_selected.fit(X_train_selected, y_train)

# Create a unique folder name dynamically
model_dir = f"Models_test/19_Feature_Tuning_{train_probe}_to_{test_probe}_ADC{adc_num}_{sample_rate}_MSPS"
os.makedirs(model_dir, exist_ok=True)

# Save the Model and Scaler
joblib.dump(rf_selected, os.path.join(model_dir, "RandomForest_Model.pkl"))
joblib.dump(scaler, os.path.join(model_dir, "Scaler.pkl"))
joblib.dump(selected_feature_indices, os.path.join(model_dir, "Selected_Features.pkl"))

# Predictions
y_pred_selected = rf_selected.predict(X_test_selected)
y_probs_selected = rf_selected.predict_proba(X_test_selected)[:, 1]

# ==================================================================
# Performance Metrics After Feature Selection
# ==================================================================
print("\nModel Performance Using 19 Features:")
print(f"Test Accuracy: {accuracy_score(y_test, y_pred_selected):.4f}")
print(f"Balanced Accuracy: {balanced_accuracy_score(y_test, y_pred_selected):.4f}")
print(f"MCC: {matthews_corrcoef(y_test, y_pred_selected):.4f}")
print(f"Log Loss: {log_loss(y_test, y_probs_selected):.4f}")
print(f"F1 Score: {f1_score(y_test, y_pred_selected):.4f}")
print(f"Recall: {recall_score(y_test, y_pred_selected):.4f}")
print(f"Precision: {precision_score(y_test, y_pred_selected):.4f}")
print(classification_report(y_test, y_pred_selected, target_names=['CH1', 'CH2']))

# Compute Confusion Matrix, FPR, and FNR
cm = confusion_matrix(y_test, y_pred_selected)
print("\nConfusion Matrix:")
print(cm)
fp_rate = cm[0][1] / (cm[0][1] + cm[0][0])
fn_rate = cm[1][0] / (cm[1][0] + cm[1][1])
print(f"False Positive Rate (FPR): {fp_rate:.4f}")
print(f"False Negative Rate (FNR): {fn_rate:.4f}")


✅ Loaded Probe1 ADC1 CH1: (3332, 38)
✅ Loaded Probe1 ADC1 CH2: (3332, 38)
✅ Loaded Probe2 ADC1 CH1: (3332, 38)
✅ Loaded Probe2 ADC1 CH2: (3331, 38)

Selected Features and Importances:
Mean_H: Importance 0.1816
RMS_H: Importance 0.1438
Mean_L: Importance 0.1637
Variance_L: Importance 0.0800
RMS_L: Importance 0.1506
Mean_Freq_H: Importance 0.0138
Entropy_L: Importance 0.0003
Max_H: Importance 0.0362
Entropy_H: Importance 0.0908
Mean Deviation_H: Importance 0.0001
Min_L: Importance 0.0000
Variance_H: Importance 0.1083
Mean_Freq_L: Importance 0.0000
Min_H: Importance 0.0309
Mean Deviation_L: Importance 0.0000
Max_L: Importance 0.0000
Std_L: Importance 0.0000
Skewness_H: Importance 0.0000
Skewness_L: Importance 0.0000

Model Performance Using 19 Features:
Test Accuracy: 0.9973
Balanced Accuracy: 0.9973
MCC: 0.9946
Log Loss: 0.0050
F1 Score: 0.9973
Recall: 0.9949
Precision: 0.9997
              precision    recall  f1-score   support

         CH1       0.99      1.00      1.00      3332
   

## Testing on the models trained from above

### ADC2 Probe3 Test at 30MSPS

In [41]:
import numpy as np
import os
import joblib
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score, classification_report, confusion_matrix, log_loss,
    f1_score, recall_score, precision_score, matthews_corrcoef, balanced_accuracy_score
)

# ==================================================================
# Fixed Configuration
# ==================================================================
base_directory = "C:\\Users\\awm21\\Documents\\Probe_Vari\\Neural_Networks\\Features_Vitis"

adc_num = 2
test_probe_dataset = 3
sample_rate = 30

test_probe = f"Probe{test_probe_dataset}"

# Directory where the model and scaler were saved
model_dir = f"Models_test/19_Feature_Tuning_Probe1_to_Probe2_ADC1_30_MSPS"

# Load the trained model and scaler
model_path = os.path.join(model_dir, "RandomForest_Model.pkl")
scaler_path = os.path.join(model_dir, "Scaler.pkl")
features_path = os.path.join(model_dir, "Selected_Features.pkl")

if not os.path.exists(model_path) or not os.path.exists(scaler_path) or not os.path.exists(features_path):
    raise FileNotFoundError("Model, scaler, or selected features file not found.")

rf_selected = joblib.load(model_path)
scaler = joblib.load(scaler_path)
selected_feature_indices = joblib.load(features_path)

# List of all 38 loaded features
all_feature_names = [
    "Max_H", "Min_H", "Mean_H", "Std_H", "Mean Deviation_H", "RMS_H", "Skewness_H", "Kurtosis_H", "Peak-to-Peak_H", "Zero Crossing Rate_H",
    "Centroid_H", "Entropy_H", "Spread_H", "Skewness_Freq_H", "Mean_Freq_H", "Kurtosis_Freq_H", "Irregularity_H", "Variance_H", "Dominant_Freq_H",
    "Max_L", "Min_L", "Mean_L", "Std_L", "Mean Deviation_L", "RMS_L", "Skewness_L", "Kurtosis_L", "Peak-to-Peak_L", "Zero Crossing Rate_L",
    "Centroid_L", "Entropy_L", "Spread_L", "Skewness_Freq_L", "Mean_Freq_L", "Kurtosis_Freq_L", "Irregularity_L", "Variance_L"
]

# Get feature names for selected features
selected_features = [all_feature_names[i] for i in selected_feature_indices]

def load_channel_data(probe_name, adc_num, channel):
    """Load specific ADC/channel data for a probe and verify ADC consistency."""
    folder = os.path.join(base_directory, f"P{probe_name[-1]}_fd")
    filename = f'fd_{probe_name}_ADC{adc_num}_CH{channel}_{sample_rate}_MSPS.npy'
    file_path = os.path.join(folder, filename)
    
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"File not found: {file_path}")
    
    data = np.load(file_path)
    print(f"✅ Loaded {probe_name} ADC{adc_num} CH{channel}: {data.shape}")
    return data

# ==================================================================
# Load Test Data
# ==================================================================
test_ch0 = load_channel_data(test_probe, adc_num, 1)
test_ch1 = load_channel_data(test_probe, adc_num, 2)

# Stack test data
X_test = np.vstack([test_ch0, test_ch1])
y_test = np.concatenate([np.zeros(test_ch0.shape[0]), np.ones(test_ch1.shape[0])])

# Apply standardization using the loaded scaler
X_test_scaled = scaler.transform(X_test)

# Select only required 19 features from the 38 loaded dataset
X_test_selected = X_test_scaled[:, selected_feature_indices]

# ==================================================================
# Evaluate the Model
# ==================================================================
y_pred_selected = rf_selected.predict(X_test_selected)
y_probs_selected = rf_selected.predict_proba(X_test_selected)[:, 1]

# Print Model Performance
print("\nModel Performance Using 19 Features:")
print(f"Test Accuracy: {accuracy_score(y_test, y_pred_selected):.4f}")
print(f"Balanced Accuracy: {balanced_accuracy_score(y_test, y_pred_selected):.4f}")
print(f"MCC: {matthews_corrcoef(y_test, y_pred_selected):.4f}")
print(f"Log Loss: {log_loss(y_test, y_probs_selected):.4f}")
print(f"F1 Score: {f1_score(y_test, y_pred_selected):.4f}")
print(f"Recall: {recall_score(y_test, y_pred_selected):.4f}")
print(f"Precision: {precision_score(y_test, y_pred_selected):.4f}")
print(classification_report(y_test, y_pred_selected, target_names=['CH1', 'CH2']))

# Compute Confusion Matrix, FPR, and FNR
cm = confusion_matrix(y_test, y_pred_selected)
print("\nConfusion Matrix:")
print(cm)
fp_rate = cm[0][1] / (cm[0][1] + cm[0][0])
fn_rate = cm[1][0] / (cm[1][0] + cm[1][1])
print(f"False Positive Rate (FPR): {fp_rate:.4f}")
print(f"False Negative Rate (FNR): {fn_rate:.4f}")


✅ Loaded Probe3 ADC2 CH1: (3331, 38)
✅ Loaded Probe3 ADC2 CH2: (3332, 38)

Model Performance Using 19 Features:
Test Accuracy: 0.8922
Balanced Accuracy: 0.8923
MCC: 0.8034
Log Loss: 0.7624
F1 Score: 0.8792
Recall: 0.7845
Precision: 1.0000
              precision    recall  f1-score   support

         CH1       0.82      1.00      0.90      3331
         CH2       1.00      0.78      0.88      3332

    accuracy                           0.89      6663
   macro avg       0.91      0.89      0.89      6663
weighted avg       0.91      0.89      0.89      6663


Confusion Matrix:
[[3331    0]
 [ 718 2614]]
False Positive Rate (FPR): 0.0000
False Negative Rate (FNR): 0.2155


### ADC3 Probe3 Test at 30MSPS

In [42]:
import numpy as np
import os
import joblib
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score, classification_report, confusion_matrix, log_loss,
    f1_score, recall_score, precision_score, matthews_corrcoef, balanced_accuracy_score
)

# ==================================================================
# Fixed Configuration
# ==================================================================
base_directory = "C:\\Users\\awm21\\Documents\\Probe_Vari\\Neural_Networks\\Features_Vitis"

adc_num = 3
test_probe_dataset = 3
sample_rate = 30

test_probe = f"Probe{test_probe_dataset}"

# Directory where the model and scaler were saved
model_dir = f"Models_test/19_Feature_Tuning_Probe1_to_Probe2_ADC1_30_MSPS"

# Load the trained model and scaler
model_path = os.path.join(model_dir, "RandomForest_Model.pkl")
scaler_path = os.path.join(model_dir, "Scaler.pkl")
features_path = os.path.join(model_dir, "Selected_Features.pkl")

if not os.path.exists(model_path) or not os.path.exists(scaler_path) or not os.path.exists(features_path):
    raise FileNotFoundError("Model, scaler, or selected features file not found.")

rf_selected = joblib.load(model_path)
scaler = joblib.load(scaler_path)
selected_feature_indices = joblib.load(features_path)

# List of all 38 loaded features
all_feature_names = [
    "Max_H", "Min_H", "Mean_H", "Std_H", "Mean Deviation_H", "RMS_H", "Skewness_H", "Kurtosis_H", "Peak-to-Peak_H", "Zero Crossing Rate_H",
    "Centroid_H", "Entropy_H", "Spread_H", "Skewness_Freq_H", "Mean_Freq_H", "Kurtosis_Freq_H", "Irregularity_H", "Variance_H", "Dominant_Freq_H",
    "Max_L", "Min_L", "Mean_L", "Std_L", "Mean Deviation_L", "RMS_L", "Skewness_L", "Kurtosis_L", "Peak-to-Peak_L", "Zero Crossing Rate_L",
    "Centroid_L", "Entropy_L", "Spread_L", "Skewness_Freq_L", "Mean_Freq_L", "Kurtosis_Freq_L", "Irregularity_L", "Variance_L"
]

# Get feature names for selected features
selected_features = [all_feature_names[i] for i in selected_feature_indices]

def load_channel_data(probe_name, adc_num, channel):
    """Load specific ADC/channel data for a probe and verify ADC consistency."""
    folder = os.path.join(base_directory, f"P{probe_name[-1]}_fd")
    filename = f'fd_{probe_name}_ADC{adc_num}_CH{channel}_{sample_rate}_MSPS.npy'
    file_path = os.path.join(folder, filename)
    
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"File not found: {file_path}")
    
    data = np.load(file_path)
    print(f"✅ Loaded {probe_name} ADC{adc_num} CH{channel}: {data.shape}")
    return data

# ==================================================================
# Load Test Data
# ==================================================================
test_ch0 = load_channel_data(test_probe, adc_num, 1)
test_ch1 = load_channel_data(test_probe, adc_num, 2)

# Stack test data
X_test = np.vstack([test_ch0, test_ch1])
y_test = np.concatenate([np.zeros(test_ch0.shape[0]), np.ones(test_ch1.shape[0])])

# Apply standardization using the loaded scaler
X_test_scaled = scaler.transform(X_test)

# Select only required 19 features from the 38 loaded dataset
X_test_selected = X_test_scaled[:, selected_feature_indices]

# ==================================================================
# Evaluate the Model
# ==================================================================
y_pred_selected = rf_selected.predict(X_test_selected)
y_probs_selected = rf_selected.predict_proba(X_test_selected)[:, 1]

# Print Model Performance
print("\nModel Performance Using 19 Features:")
print(f"Test Accuracy: {accuracy_score(y_test, y_pred_selected):.4f}")
print(f"Balanced Accuracy: {balanced_accuracy_score(y_test, y_pred_selected):.4f}")
print(f"MCC: {matthews_corrcoef(y_test, y_pred_selected):.4f}")
print(f"Log Loss: {log_loss(y_test, y_probs_selected):.4f}")
print(f"F1 Score: {f1_score(y_test, y_pred_selected):.4f}")
print(f"Recall: {recall_score(y_test, y_pred_selected):.4f}")
print(f"Precision: {precision_score(y_test, y_pred_selected):.4f}")
print(classification_report(y_test, y_pred_selected, target_names=['CH1', 'CH2']))

# Compute Confusion Matrix, FPR, and FNR
cm = confusion_matrix(y_test, y_pred_selected)
print("\nConfusion Matrix:")
print(cm)
fp_rate = cm[0][1] / (cm[0][1] + cm[0][0])
fn_rate = cm[1][0] / (cm[1][0] + cm[1][1])
print(f"False Positive Rate (FPR): {fp_rate:.4f}")
print(f"False Negative Rate (FNR): {fn_rate:.4f}")


✅ Loaded Probe3 ADC3 CH1: (3332, 38)
✅ Loaded Probe3 ADC3 CH2: (3331, 38)

Model Performance Using 19 Features:
Test Accuracy: 0.9958
Balanced Accuracy: 0.9958
MCC: 0.9916
Log Loss: 0.0151
F1 Score: 0.9958
Recall: 0.9916
Precision: 1.0000
              precision    recall  f1-score   support

         CH1       0.99      1.00      1.00      3332
         CH2       1.00      0.99      1.00      3331

    accuracy                           1.00      6663
   macro avg       1.00      1.00      1.00      6663
weighted avg       1.00      1.00      1.00      6663


Confusion Matrix:
[[3332    0]
 [  28 3303]]
False Positive Rate (FPR): 0.0000
False Negative Rate (FNR): 0.0084


# 40 MSPS

## Training the model with 19 Features


In [11]:
import numpy as np
import os
import joblib
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import skew, kurtosis
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score, classification_report, confusion_matrix,
    roc_curve, auc, precision_recall_curve, matthews_corrcoef, 
    balanced_accuracy_score, log_loss, f1_score, recall_score, precision_score
)

# ==================================================================
# Fixed Configuration
# ==================================================================
base_directory = "C:\\Users\\awm21\\Documents\\Probe_Vari\\Neural_Networks\\Features_Vitis"

adc_num = 1
train_probe_dataset = 1
test_probe_dataset = 2
sample_rate = 40

train_probe = f"Probe{train_probe_dataset}"
test_probe = f"Probe{test_probe_dataset}"

# List of all 38 loaded features
all_feature_names = [
    "Max_H", "Min_H", "Mean_H", "Std_H", "Mean Deviation_H", "RMS_H", "Skewness_H", "Kurtosis_H", "Peak-to-Peak_H", "Zero Crossing Rate_H",
    "Centroid_H", "Entropy_H", "Spread_H", "Skewness_Freq_H", "Mean_Freq_H", "Kurtosis_Freq_H", "Irregularity_H", "Variance_H", "Dominant_Freq_H",
    "Max_L", "Min_L", "Mean_L", "Std_L", "Mean Deviation_L", "RMS_L", "Skewness_L", "Kurtosis_L", "Peak-to-Peak_L", "Zero Crossing Rate_L",
    "Centroid_L", "Entropy_L", "Spread_L", "Skewness_Freq_L", "Mean_Freq_L", "Kurtosis_Freq_L", "Irregularity_L", "Variance_L"
]

# Select only the 19 required features from the 38 loaded features
selected_features = [
    "Mean_H", "RMS_H", "Mean_L", "Variance_L", "RMS_L", "Mean_Freq_H", "Entropy_L", 
    "Max_H", "Entropy_H", "Mean Deviation_H", "Min_L", "Variance_H", "Mean_Freq_L", 
    "Min_H", "Mean Deviation_L", "Max_L", "Std_L", "Skewness_H", "Skewness_L"
]

# Get indices of selected features in the original dataset
selected_feature_indices = [all_feature_names.index(f) for f in selected_features]

def load_channel_data(probe_name, adc_num, channel):
    """Load specific ADC/channel data for a probe and verify ADC consistency."""
    folder = os.path.join(base_directory, f"P{probe_name[-1]}_fd")
    filename = f'fd_{probe_name}_ADC{adc_num}_CH{channel}_{sample_rate}_MSPS.npy'
    file_path = os.path.join(folder, filename)
    
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"File not found: {file_path}")
    
    data = np.load(file_path)
    print(f"✅ Loaded {probe_name} ADC{adc_num} CH{channel}: {data.shape}")
    return data

# ==================================================================
# Training & Test Data
# ==================================================================
train_ch0 = load_channel_data(train_probe, adc_num, 1)
train_ch1 = load_channel_data(train_probe, adc_num, 2)
test_ch0 = load_channel_data(test_probe, adc_num, 1)
test_ch1 = load_channel_data(test_probe, adc_num, 2)

# Stack training and test data
X_train = np.vstack([train_ch0, train_ch1])
y_train = np.concatenate([np.zeros(train_ch0.shape[0]), np.ones(train_ch1.shape[0])])
X_test = np.vstack([test_ch0, test_ch1])
y_test = np.concatenate([np.zeros(test_ch0.shape[0]), np.ones(test_ch1.shape[0])])

# ==================================================================
# Data Preprocessing
# ==================================================================
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Select only required 19 features from the 38 loaded dataset
X_train_selected = X_train_scaled[:, selected_feature_indices]
X_test_selected = X_test_scaled[:, selected_feature_indices]

# ==================================================================
# Train Model with Selected Features (19 Features)
# ==================================================================
rf = RandomForestClassifier(
    n_estimators=50,
    max_depth=10,
    min_samples_split=10,
    min_samples_leaf=5,
    max_features='sqrt',
    random_state=42
)
rf.fit(X_train_selected, y_train)

# Get feature importance only for selected features
feature_importances = rf.feature_importances_

# Print Selected Features and their Importances
print("\nSelected Features and Importances:")
for feature_idx, feature_name in zip(range(len(selected_features)), selected_features):
    print(f"{feature_name}: Importance {feature_importances[feature_idx]:.4f}")

# ==================================================================
# Retrain Model with Selected Features
# ==================================================================
rf_selected = RandomForestClassifier(
    n_estimators=50,
    max_depth=10,
    min_samples_split=10,
    min_samples_leaf=5,
    max_features='sqrt',
    random_state=42
)
rf_selected.fit(X_train_selected, y_train)

# Create a unique folder name dynamically
model_dir = f"Models_test/19_Feature_Tuning_{train_probe}_to_{test_probe}_ADC{adc_num}_{sample_rate}_MSPS"
os.makedirs(model_dir, exist_ok=True)

# Save the Model and Scaler
joblib.dump(rf_selected, os.path.join(model_dir, "RandomForest_Model.pkl"))
joblib.dump(scaler, os.path.join(model_dir, "Scaler.pkl"))
joblib.dump(selected_feature_indices, os.path.join(model_dir, "Selected_Features.pkl"))

# Predictions
y_pred_selected = rf_selected.predict(X_test_selected)
y_probs_selected = rf_selected.predict_proba(X_test_selected)[:, 1]

# ==================================================================
# Performance Metrics After Feature Selection
# ==================================================================
print("\nModel Performance Using 19 Features:")
print(f"Test Accuracy: {accuracy_score(y_test, y_pred_selected):.4f}")
print(f"Balanced Accuracy: {balanced_accuracy_score(y_test, y_pred_selected):.4f}")
print(f"MCC: {matthews_corrcoef(y_test, y_pred_selected):.4f}")
print(f"Log Loss: {log_loss(y_test, y_probs_selected):.4f}")
print(f"F1 Score: {f1_score(y_test, y_pred_selected):.4f}")
print(f"Recall: {recall_score(y_test, y_pred_selected):.4f}")
print(f"Precision: {precision_score(y_test, y_pred_selected):.4f}")
print(classification_report(y_test, y_pred_selected, target_names=['CH1', 'CH2']))

# Compute Confusion Matrix, FPR, and FNR
cm = confusion_matrix(y_test, y_pred_selected)
print("\nConfusion Matrix:")
print(cm)
fp_rate = cm[0][1] / (cm[0][1] + cm[0][0])
fn_rate = cm[1][0] / (cm[1][0] + cm[1][1])
print(f"False Positive Rate (FPR): {fp_rate:.4f}")
print(f"False Negative Rate (FNR): {fn_rate:.4f}")


✅ Loaded Probe1 ADC1 CH1: (2481, 38)
✅ Loaded Probe1 ADC1 CH2: (2481, 38)
✅ Loaded Probe2 ADC1 CH1: (2481, 38)
✅ Loaded Probe2 ADC1 CH2: (2481, 38)

Selected Features and Importances:
Mean_H: Importance 0.1860
RMS_H: Importance 0.1447
Mean_L: Importance 0.1713
Variance_L: Importance 0.0800
RMS_L: Importance 0.1565
Mean_Freq_H: Importance 0.0142
Entropy_L: Importance 0.0305
Max_H: Importance 0.0394
Entropy_H: Importance 0.0290
Mean Deviation_H: Importance 0.0005
Min_L: Importance 0.0000
Variance_H: Importance 0.1050
Mean_Freq_L: Importance 0.0000
Min_H: Importance 0.0287
Mean Deviation_L: Importance 0.0000
Max_L: Importance 0.0142
Std_L: Importance 0.0000
Skewness_H: Importance 0.0000
Skewness_L: Importance 0.0000

Model Performance Using 19 Features:
Test Accuracy: 1.0000
Balanced Accuracy: 1.0000
MCC: 1.0000
Log Loss: 0.0004
F1 Score: 1.0000
Recall: 1.0000
Precision: 1.0000
              precision    recall  f1-score   support

         CH1       1.00      1.00      1.00      2481
   

## Testing on the models trained from above


### ADC2 Probe3 Test at 40MSPS

In [38]:
import numpy as np
import os
import joblib
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score, classification_report, confusion_matrix, log_loss,
    f1_score, recall_score, precision_score, matthews_corrcoef, balanced_accuracy_score
)

# ==================================================================
# Fixed Configuration
# ==================================================================
base_directory = "C:\\Users\\awm21\\Documents\\Probe_Vari\\Neural_Networks\\Features_Vitis"

adc_num = 2
test_probe_dataset = 3
sample_rate = 40

test_probe = f"Probe{test_probe_dataset}"

# Directory where the model and scaler were saved
model_dir = f"Models_test/19_Feature_Tuning_Probe1_to_Probe2_ADC1_40_MSPS"

# Load the trained model and scaler
model_path = os.path.join(model_dir, "RandomForest_Model.pkl")
scaler_path = os.path.join(model_dir, "Scaler.pkl")
features_path = os.path.join(model_dir, "Selected_Features.pkl")

if not os.path.exists(model_path) or not os.path.exists(scaler_path) or not os.path.exists(features_path):
    raise FileNotFoundError("Model, scaler, or selected features file not found.")

rf_selected = joblib.load(model_path)
scaler = joblib.load(scaler_path)
selected_feature_indices = joblib.load(features_path)

# List of all 38 loaded features
all_feature_names = [
    "Max_H", "Min_H", "Mean_H", "Std_H", "Mean Deviation_H", "RMS_H", "Skewness_H", "Kurtosis_H", "Peak-to-Peak_H", "Zero Crossing Rate_H",
    "Centroid_H", "Entropy_H", "Spread_H", "Skewness_Freq_H", "Mean_Freq_H", "Kurtosis_Freq_H", "Irregularity_H", "Variance_H", "Dominant_Freq_H",
    "Max_L", "Min_L", "Mean_L", "Std_L", "Mean Deviation_L", "RMS_L", "Skewness_L", "Kurtosis_L", "Peak-to-Peak_L", "Zero Crossing Rate_L",
    "Centroid_L", "Entropy_L", "Spread_L", "Skewness_Freq_L", "Mean_Freq_L", "Kurtosis_Freq_L", "Irregularity_L", "Variance_L"
]

# Get feature names for selected features
selected_features = [all_feature_names[i] for i in selected_feature_indices]

def load_channel_data(probe_name, adc_num, channel):
    """Load specific ADC/channel data for a probe and verify ADC consistency."""
    folder = os.path.join(base_directory, f"P{probe_name[-1]}_fd")
    filename = f'fd_{probe_name}_ADC{adc_num}_CH{channel}_{sample_rate}_MSPS.npy'
    file_path = os.path.join(folder, filename)
    
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"File not found: {file_path}")
    
    data = np.load(file_path)
    print(f"✅ Loaded {probe_name} ADC{adc_num} CH{channel}: {data.shape}")
    return data

# ==================================================================
# Load Test Data
# ==================================================================
test_ch0 = load_channel_data(test_probe, adc_num, 1)
test_ch1 = load_channel_data(test_probe, adc_num, 2)

# Stack test data
X_test = np.vstack([test_ch0, test_ch1])
y_test = np.concatenate([np.zeros(test_ch0.shape[0]), np.ones(test_ch1.shape[0])])

# Apply standardization using the loaded scaler
X_test_scaled = scaler.transform(X_test)

# Select only required 19 features from the 38 loaded dataset
X_test_selected = X_test_scaled[:, selected_feature_indices]

# ==================================================================
# Evaluate the Model
# ==================================================================
y_pred_selected = rf_selected.predict(X_test_selected)
y_probs_selected = rf_selected.predict_proba(X_test_selected)[:, 1]

# Print Model Performance
print("\nModel Performance Using 19 Features:")
print(f"Test Accuracy: {accuracy_score(y_test, y_pred_selected):.4f}")
print(f"Balanced Accuracy: {balanced_accuracy_score(y_test, y_pred_selected):.4f}")
print(f"MCC: {matthews_corrcoef(y_test, y_pred_selected):.4f}")
print(f"Log Loss: {log_loss(y_test, y_probs_selected):.4f}")
print(f"F1 Score: {f1_score(y_test, y_pred_selected):.4f}")
print(f"Recall: {recall_score(y_test, y_pred_selected):.4f}")
print(f"Precision: {precision_score(y_test, y_pred_selected):.4f}")
print(classification_report(y_test, y_pred_selected, target_names=['CH1', 'CH2']))

# Compute Confusion Matrix, FPR, and FNR
cm = confusion_matrix(y_test, y_pred_selected)
print("\nConfusion Matrix:")
print(cm)
fp_rate = cm[0][1] / (cm[0][1] + cm[0][0])
fn_rate = cm[1][0] / (cm[1][0] + cm[1][1])
print(f"False Positive Rate (FPR): {fp_rate:.4f}")
print(f"False Negative Rate (FNR): {fn_rate:.4f}")


✅ Loaded Probe3 ADC2 CH1: (2481, 38)
✅ Loaded Probe3 ADC2 CH2: (2481, 38)

Model Performance Using 19 Features:
Test Accuracy: 1.0000
Balanced Accuracy: 1.0000
MCC: 1.0000
Log Loss: 0.0037
F1 Score: 1.0000
Recall: 1.0000
Precision: 1.0000
              precision    recall  f1-score   support

         CH1       1.00      1.00      1.00      2481
         CH2       1.00      1.00      1.00      2481

    accuracy                           1.00      4962
   macro avg       1.00      1.00      1.00      4962
weighted avg       1.00      1.00      1.00      4962


Confusion Matrix:
[[2481    0]
 [   0 2481]]
False Positive Rate (FPR): 0.0000
False Negative Rate (FNR): 0.0000


### ADC3 Probe3 Test at 40MSPS

In [37]:
import numpy as np
import os
import joblib
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score, classification_report, confusion_matrix, log_loss,
    f1_score, recall_score, precision_score, matthews_corrcoef, balanced_accuracy_score
)

# ==================================================================
# Fixed Configuration
# ==================================================================
base_directory = "C:\\Users\\awm21\\Documents\\Probe_Vari\\Neural_Networks\\Features_Vitis"

adc_num = 3
test_probe_dataset = 3
sample_rate = 40

test_probe = f"Probe{test_probe_dataset}"

# Directory where the model and scaler were saved
model_dir = f"Models_test/19_Feature_Tuning_Probe1_to_Probe2_ADC1_40_MSPS"

# Load the trained model and scaler
model_path = os.path.join(model_dir, "RandomForest_Model.pkl")
scaler_path = os.path.join(model_dir, "Scaler.pkl")
features_path = os.path.join(model_dir, "Selected_Features.pkl")

if not os.path.exists(model_path) or not os.path.exists(scaler_path) or not os.path.exists(features_path):
    raise FileNotFoundError("Model, scaler, or selected features file not found.")

rf_selected = joblib.load(model_path)
scaler = joblib.load(scaler_path)
selected_feature_indices = joblib.load(features_path)

# List of all 38 loaded features
all_feature_names = [
    "Max_H", "Min_H", "Mean_H", "Std_H", "Mean Deviation_H", "RMS_H", "Skewness_H", "Kurtosis_H", "Peak-to-Peak_H", "Zero Crossing Rate_H",
    "Centroid_H", "Entropy_H", "Spread_H", "Skewness_Freq_H", "Mean_Freq_H", "Kurtosis_Freq_H", "Irregularity_H", "Variance_H", "Dominant_Freq_H",
    "Max_L", "Min_L", "Mean_L", "Std_L", "Mean Deviation_L", "RMS_L", "Skewness_L", "Kurtosis_L", "Peak-to-Peak_L", "Zero Crossing Rate_L",
    "Centroid_L", "Entropy_L", "Spread_L", "Skewness_Freq_L", "Mean_Freq_L", "Kurtosis_Freq_L", "Irregularity_L", "Variance_L"
]

# Get feature names for selected features
selected_features = [all_feature_names[i] for i in selected_feature_indices]

def load_channel_data(probe_name, adc_num, channel):
    """Load specific ADC/channel data for a probe and verify ADC consistency."""
    folder = os.path.join(base_directory, f"P{probe_name[-1]}_fd")
    filename = f'fd_{probe_name}_ADC{adc_num}_CH{channel}_{sample_rate}_MSPS.npy'
    file_path = os.path.join(folder, filename)
    
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"File not found: {file_path}")
    
    data = np.load(file_path)
    print(f"✅ Loaded {probe_name} ADC{adc_num} CH{channel}: {data.shape}")
    return data

# ==================================================================
# Load Test Data
# ==================================================================
test_ch0 = load_channel_data(test_probe, adc_num, 1)
test_ch1 = load_channel_data(test_probe, adc_num, 2)

# Stack test data
X_test = np.vstack([test_ch0, test_ch1])
y_test = np.concatenate([np.zeros(test_ch0.shape[0]), np.ones(test_ch1.shape[0])])

# Apply standardization using the loaded scaler
X_test_scaled = scaler.transform(X_test)

# Select only required 19 features from the 38 loaded dataset
X_test_selected = X_test_scaled[:, selected_feature_indices]

# ==================================================================
# Evaluate the Model
# ==================================================================
y_pred_selected = rf_selected.predict(X_test_selected)
y_probs_selected = rf_selected.predict_proba(X_test_selected)[:, 1]

# Print Model Performance
print("\nModel Performance Using 19 Features:")
print(f"Test Accuracy: {accuracy_score(y_test, y_pred_selected):.4f}")
print(f"Balanced Accuracy: {balanced_accuracy_score(y_test, y_pred_selected):.4f}")
print(f"MCC: {matthews_corrcoef(y_test, y_pred_selected):.4f}")
print(f"Log Loss: {log_loss(y_test, y_probs_selected):.4f}")
print(f"F1 Score: {f1_score(y_test, y_pred_selected):.4f}")
print(f"Recall: {recall_score(y_test, y_pred_selected):.4f}")
print(f"Precision: {precision_score(y_test, y_pred_selected):.4f}")
print(classification_report(y_test, y_pred_selected, target_names=['CH1', 'CH2']))

# Compute Confusion Matrix, FPR, and FNR
cm = confusion_matrix(y_test, y_pred_selected)
print("\nConfusion Matrix:")
print(cm)
fp_rate = cm[0][1] / (cm[0][1] + cm[0][0])
fn_rate = cm[1][0] / (cm[1][0] + cm[1][1])
print(f"False Positive Rate (FPR): {fp_rate:.4f}")
print(f"False Negative Rate (FNR): {fn_rate:.4f}")


✅ Loaded Probe3 ADC3 CH1: (2481, 38)
✅ Loaded Probe3 ADC3 CH2: (2481, 38)

Model Performance Using 19 Features:
Test Accuracy: 1.0000
Balanced Accuracy: 1.0000
MCC: 1.0000
Log Loss: 0.0038
F1 Score: 1.0000
Recall: 1.0000
Precision: 1.0000
              precision    recall  f1-score   support

         CH1       1.00      1.00      1.00      2481
         CH2       1.00      1.00      1.00      2481

    accuracy                           1.00      4962
   macro avg       1.00      1.00      1.00      4962
weighted avg       1.00      1.00      1.00      4962


Confusion Matrix:
[[2481    0]
 [   0 2481]]
False Positive Rate (FPR): 0.0000
False Negative Rate (FNR): 0.0000


# 50 MSPS

## Training the model with 19 Features

In [15]:
import numpy as np
import os
import joblib
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import skew, kurtosis
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score, classification_report, confusion_matrix,
    roc_curve, auc, precision_recall_curve, matthews_corrcoef, 
    balanced_accuracy_score, log_loss, f1_score, recall_score, precision_score
)

# ==================================================================
# Fixed Configuration
# ==================================================================
base_directory = "C:\\Users\\awm21\\Documents\\Probe_Vari\\Neural_Networks\\Features_Vitis"

adc_num = 1
train_probe_dataset = 1
test_probe_dataset = 2
sample_rate = 50

train_probe = f"Probe{train_probe_dataset}"
test_probe = f"Probe{test_probe_dataset}"

# List of all 38 loaded features
all_feature_names = [
    "Max_H", "Min_H", "Mean_H", "Std_H", "Mean Deviation_H", "RMS_H", "Skewness_H", "Kurtosis_H", "Peak-to-Peak_H", "Zero Crossing Rate_H",
    "Centroid_H", "Entropy_H", "Spread_H", "Skewness_Freq_H", "Mean_Freq_H", "Kurtosis_Freq_H", "Irregularity_H", "Variance_H", "Dominant_Freq_H",
    "Max_L", "Min_L", "Mean_L", "Std_L", "Mean Deviation_L", "RMS_L", "Skewness_L", "Kurtosis_L", "Peak-to-Peak_L", "Zero Crossing Rate_L",
    "Centroid_L", "Entropy_L", "Spread_L", "Skewness_Freq_L", "Mean_Freq_L", "Kurtosis_Freq_L", "Irregularity_L", "Variance_L"
]

# Select only the 19 required features from the 38 loaded features
selected_features = [
    "Mean_H", "RMS_H", "Mean_L", "Variance_L", "RMS_L", "Mean_Freq_H", "Entropy_L", 
    "Max_H", "Entropy_H", "Mean Deviation_H", "Min_L", "Variance_H", "Mean_Freq_L", 
    "Min_H", "Mean Deviation_L", "Max_L", "Std_L", "Skewness_H", "Skewness_L"
]

# Get indices of selected features in the original dataset
selected_feature_indices = [all_feature_names.index(f) for f in selected_features]

def load_channel_data(probe_name, adc_num, channel):
    """Load specific ADC/channel data for a probe and verify ADC consistency."""
    folder = os.path.join(base_directory, f"P{probe_name[-1]}_fd")
    filename = f'fd_{probe_name}_ADC{adc_num}_CH{channel}_{sample_rate}_MSPS.npy'
    file_path = os.path.join(folder, filename)
    
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"File not found: {file_path}")
    
    data = np.load(file_path)
    print(f"✅ Loaded {probe_name} ADC{adc_num} CH{channel}: {data.shape}")
    return data

# ==================================================================
# Training & Test Data
# ==================================================================
train_ch0 = load_channel_data(train_probe, adc_num, 1)
train_ch1 = load_channel_data(train_probe, adc_num, 2)
test_ch0 = load_channel_data(test_probe, adc_num, 1)
test_ch1 = load_channel_data(test_probe, adc_num, 2)

# Stack training and test data
X_train = np.vstack([train_ch0, train_ch1])
y_train = np.concatenate([np.zeros(train_ch0.shape[0]), np.ones(train_ch1.shape[0])])
X_test = np.vstack([test_ch0, test_ch1])
y_test = np.concatenate([np.zeros(test_ch0.shape[0]), np.ones(test_ch1.shape[0])])

# ==================================================================
# Data Preprocessing
# ==================================================================
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Select only required 19 features from the 38 loaded dataset
X_train_selected = X_train_scaled[:, selected_feature_indices]
X_test_selected = X_test_scaled[:, selected_feature_indices]

# ==================================================================
# Train Model with Selected Features (19 Features)
# ==================================================================
rf = RandomForestClassifier(
    n_estimators=50,
    max_depth=10,
    min_samples_split=10,
    min_samples_leaf=5,
    max_features='sqrt',
    random_state=42
)
rf.fit(X_train_selected, y_train)

# Get feature importance only for selected features
feature_importances = rf.feature_importances_

# Print Selected Features and their Importances
print("\nSelected Features and Importances:")
for feature_idx, feature_name in zip(range(len(selected_features)), selected_features):
    print(f"{feature_name}: Importance {feature_importances[feature_idx]:.4f}")

# ==================================================================
# Retrain Model with Selected Features
# ==================================================================
rf_selected = RandomForestClassifier(
    n_estimators=50,
    max_depth=10,
    min_samples_split=10,
    min_samples_leaf=5,
    max_features='sqrt',
    random_state=42
)
rf_selected.fit(X_train_selected, y_train)

# Create a unique folder name dynamically
model_dir = f"Models_test/19_Feature_Tuning_{train_probe}_to_{test_probe}_ADC{adc_num}_{sample_rate}_MSPS"
os.makedirs(model_dir, exist_ok=True)

# Save the Model and Scaler
joblib.dump(rf_selected, os.path.join(model_dir, "RandomForest_Model.pkl"))
joblib.dump(scaler, os.path.join(model_dir, "Scaler.pkl"))
joblib.dump(selected_feature_indices, os.path.join(model_dir, "Selected_Features.pkl"))

# Predictions
y_pred_selected = rf_selected.predict(X_test_selected)
y_probs_selected = rf_selected.predict_proba(X_test_selected)[:, 1]

# ==================================================================
# Performance Metrics After Feature Selection
# ==================================================================
print("\nModel Performance Using 19 Features:")
print(f"Test Accuracy: {accuracy_score(y_test, y_pred_selected):.4f}")
print(f"Balanced Accuracy: {balanced_accuracy_score(y_test, y_pred_selected):.4f}")
print(f"MCC: {matthews_corrcoef(y_test, y_pred_selected):.4f}")
print(f"Log Loss: {log_loss(y_test, y_probs_selected):.4f}")
print(f"F1 Score: {f1_score(y_test, y_pred_selected):.4f}")
print(f"Recall: {recall_score(y_test, y_pred_selected):.4f}")
print(f"Precision: {precision_score(y_test, y_pred_selected):.4f}")
print(classification_report(y_test, y_pred_selected, target_names=['CH1', 'CH2']))

# Compute Confusion Matrix, FPR, and FNR
cm = confusion_matrix(y_test, y_pred_selected)
print("\nConfusion Matrix:")
print(cm)
fp_rate = cm[0][1] / (cm[0][1] + cm[0][0])
fn_rate = cm[1][0] / (cm[1][0] + cm[1][1])
print(f"False Positive Rate (FPR): {fp_rate:.4f}")
print(f"False Negative Rate (FNR): {fn_rate:.4f}")


✅ Loaded Probe1 ADC1 CH1: (1998, 38)
✅ Loaded Probe1 ADC1 CH2: (1998, 38)
✅ Loaded Probe2 ADC1 CH1: (1998, 38)
✅ Loaded Probe2 ADC1 CH2: (1998, 38)

Selected Features and Importances:
Mean_H: Importance 0.1800
RMS_H: Importance 0.1200
Mean_L: Importance 0.1000
Variance_L: Importance 0.0800
RMS_L: Importance 0.1000
Mean_Freq_H: Importance 0.1800
Entropy_L: Importance 0.0196
Max_H: Importance 0.0399
Entropy_H: Importance 0.1001
Mean Deviation_H: Importance 0.0000
Min_L: Importance 0.0000
Variance_H: Importance 0.0803
Mean_Freq_L: Importance 0.0000
Min_H: Importance 0.0000
Mean Deviation_L: Importance 0.0000
Max_L: Importance 0.0000
Std_L: Importance 0.0000
Skewness_H: Importance 0.0000
Skewness_L: Importance 0.0000

Model Performance Using 19 Features:
Test Accuracy: 1.0000
Balanced Accuracy: 1.0000
MCC: 1.0000
Log Loss: 0.0416
F1 Score: 1.0000
Recall: 1.0000
Precision: 1.0000
              precision    recall  f1-score   support

         CH1       1.00      1.00      1.00      1998
   

## Testing on the models trained from above


### ADC2 Probe3 Test at 50 MSPS

In [36]:

import numpy as np
import os
import joblib
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score, classification_report, confusion_matrix, log_loss,
    f1_score, recall_score, precision_score, matthews_corrcoef, balanced_accuracy_score
)

# ==================================================================
# Fixed Configuration
# ==================================================================
base_directory = "C:\\Users\\awm21\\Documents\\Probe_Vari\\Neural_Networks\\Features_Vitis"

adc_num = 2
test_probe_dataset = 3
sample_rate = 50

test_probe = f"Probe{test_probe_dataset}"

# Directory where the model and scaler were saved
model_dir = f"Models_test/19_Feature_Tuning_Probe1_to_Probe2_ADC1_50_MSPS"

# Load the trained model and scaler
model_path = os.path.join(model_dir, "RandomForest_Model.pkl")
scaler_path = os.path.join(model_dir, "Scaler.pkl")
features_path = os.path.join(model_dir, "Selected_Features.pkl")

if not os.path.exists(model_path) or not os.path.exists(scaler_path) or not os.path.exists(features_path):
    raise FileNotFoundError("Model, scaler, or selected features file not found.")

rf_selected = joblib.load(model_path)
scaler = joblib.load(scaler_path)
selected_feature_indices = joblib.load(features_path)

# List of all 38 loaded features
all_feature_names = [
    "Max_H", "Min_H", "Mean_H", "Std_H", "Mean Deviation_H", "RMS_H", "Skewness_H", "Kurtosis_H", "Peak-to-Peak_H", "Zero Crossing Rate_H",
    "Centroid_H", "Entropy_H", "Spread_H", "Skewness_Freq_H", "Mean_Freq_H", "Kurtosis_Freq_H", "Irregularity_H", "Variance_H", "Dominant_Freq_H",
    "Max_L", "Min_L", "Mean_L", "Std_L", "Mean Deviation_L", "RMS_L", "Skewness_L", "Kurtosis_L", "Peak-to-Peak_L", "Zero Crossing Rate_L",
    "Centroid_L", "Entropy_L", "Spread_L", "Skewness_Freq_L", "Mean_Freq_L", "Kurtosis_Freq_L", "Irregularity_L", "Variance_L"
]

# Get feature names for selected features
selected_features = [all_feature_names[i] for i in selected_feature_indices]

def load_channel_data(probe_name, adc_num, channel):
    """Load specific ADC/channel data for a probe and verify ADC consistency."""
    folder = os.path.join(base_directory, f"P{probe_name[-1]}_fd")
    filename = f'fd_{probe_name}_ADC{adc_num}_CH{channel}_{sample_rate}_MSPS.npy'
    file_path = os.path.join(folder, filename)
    
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"File not found: {file_path}")
    
    data = np.load(file_path)
    print(f"✅ Loaded {probe_name} ADC{adc_num} CH{channel}: {data.shape}")
    return data

# ==================================================================
# Load Test Data
# ==================================================================
test_ch0 = load_channel_data(test_probe, adc_num, 1)
test_ch1 = load_channel_data(test_probe, adc_num, 2)

# Stack test data
X_test = np.vstack([test_ch0, test_ch1])
y_test = np.concatenate([np.zeros(test_ch0.shape[0]), np.ones(test_ch1.shape[0])])

# Apply standardization using the loaded scaler
X_test_scaled = scaler.transform(X_test)

# Select only required 19 features from the 38 loaded dataset
X_test_selected = X_test_scaled[:, selected_feature_indices]

# ==================================================================
# Evaluate the Model
# ==================================================================
y_pred_selected = rf_selected.predict(X_test_selected)
y_probs_selected = rf_selected.predict_proba(X_test_selected)[:, 1]

# Print Model Performance
print("\nModel Performance Using 19 Features:")
print(f"Test Accuracy: {accuracy_score(y_test, y_pred_selected):.4f}")
print(f"Balanced Accuracy: {balanced_accuracy_score(y_test, y_pred_selected):.4f}")
print(f"MCC: {matthews_corrcoef(y_test, y_pred_selected):.4f}")
print(f"Log Loss: {log_loss(y_test, y_probs_selected):.4f}")
print(f"F1 Score: {f1_score(y_test, y_pred_selected):.4f}")
print(f"Recall: {recall_score(y_test, y_pred_selected):.4f}")
print(f"Precision: {precision_score(y_test, y_pred_selected):.4f}")
print(classification_report(y_test, y_pred_selected, target_names=['CH1', 'CH2']))

# Compute Confusion Matrix, FPR, and FNR
cm = confusion_matrix(y_test, y_pred_selected)
print("\nConfusion Matrix:")
print(cm)
fp_rate = cm[0][1] / (cm[0][1] + cm[0][0])
fn_rate = cm[1][0] / (cm[1][0] + cm[1][1])
print(f"False Positive Rate (FPR): {fp_rate:.4f}")
print(f"False Negative Rate (FNR): {fn_rate:.4f}")


✅ Loaded Probe3 ADC2 CH1: (1998, 38)
✅ Loaded Probe3 ADC2 CH2: (1998, 38)

Model Performance Using 19 Features:
Test Accuracy: 0.9992
Balanced Accuracy: 0.9992
MCC: 0.9985
Log Loss: 0.0402
F1 Score: 0.9992
Recall: 0.9985
Precision: 1.0000
              precision    recall  f1-score   support

         CH1       1.00      1.00      1.00      1998
         CH2       1.00      1.00      1.00      1998

    accuracy                           1.00      3996
   macro avg       1.00      1.00      1.00      3996
weighted avg       1.00      1.00      1.00      3996


Confusion Matrix:
[[1998    0]
 [   3 1995]]
False Positive Rate (FPR): 0.0000
False Negative Rate (FNR): 0.0015


### ADC3 Probe3 Test at 50MSPS

In [35]:

import numpy as np
import os
import joblib
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score, classification_report, confusion_matrix, log_loss,
    f1_score, recall_score, precision_score, matthews_corrcoef, balanced_accuracy_score
)

# ==================================================================
# Fixed Configuration
# ==================================================================
base_directory = "C:\\Users\\awm21\\Documents\\Probe_Vari\\Neural_Networks\\Features_Vitis"

adc_num = 3
test_probe_dataset = 3
sample_rate = 50

test_probe = f"Probe{test_probe_dataset}"

# Directory where the model and scaler were saved
model_dir = f"Models_test/19_Feature_Tuning_Probe1_to_Probe2_ADC1_50_MSPS"

# Load the trained model and scaler
model_path = os.path.join(model_dir, "RandomForest_Model.pkl")
scaler_path = os.path.join(model_dir, "Scaler.pkl")
features_path = os.path.join(model_dir, "Selected_Features.pkl")

if not os.path.exists(model_path) or not os.path.exists(scaler_path) or not os.path.exists(features_path):
    raise FileNotFoundError("Model, scaler, or selected features file not found.")

rf_selected = joblib.load(model_path)
scaler = joblib.load(scaler_path)
selected_feature_indices = joblib.load(features_path)

# List of all 38 loaded features
all_feature_names = [
    "Max_H", "Min_H", "Mean_H", "Std_H", "Mean Deviation_H", "RMS_H", "Skewness_H", "Kurtosis_H", "Peak-to-Peak_H", "Zero Crossing Rate_H",
    "Centroid_H", "Entropy_H", "Spread_H", "Skewness_Freq_H", "Mean_Freq_H", "Kurtosis_Freq_H", "Irregularity_H", "Variance_H", "Dominant_Freq_H",
    "Max_L", "Min_L", "Mean_L", "Std_L", "Mean Deviation_L", "RMS_L", "Skewness_L", "Kurtosis_L", "Peak-to-Peak_L", "Zero Crossing Rate_L",
    "Centroid_L", "Entropy_L", "Spread_L", "Skewness_Freq_L", "Mean_Freq_L", "Kurtosis_Freq_L", "Irregularity_L", "Variance_L"
]

# Get feature names for selected features
selected_features = [all_feature_names[i] for i in selected_feature_indices]

def load_channel_data(probe_name, adc_num, channel):
    """Load specific ADC/channel data for a probe and verify ADC consistency."""
    folder = os.path.join(base_directory, f"P{probe_name[-1]}_fd")
    filename = f'fd_{probe_name}_ADC{adc_num}_CH{channel}_{sample_rate}_MSPS.npy'
    file_path = os.path.join(folder, filename)
    
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"File not found: {file_path}")
    
    data = np.load(file_path)
    print(f"✅ Loaded {probe_name} ADC{adc_num} CH{channel}: {data.shape}")
    return data

# ==================================================================
# Load Test Data
# ==================================================================
test_ch0 = load_channel_data(test_probe, adc_num, 1)
test_ch1 = load_channel_data(test_probe, adc_num, 2)

# Stack test data
X_test = np.vstack([test_ch0, test_ch1])
y_test = np.concatenate([np.zeros(test_ch0.shape[0]), np.ones(test_ch1.shape[0])])

# Apply standardization using the loaded scaler
X_test_scaled = scaler.transform(X_test)

# Select only required 19 features from the 38 loaded dataset
X_test_selected = X_test_scaled[:, selected_feature_indices]

# ==================================================================
# Evaluate the Model
# ==================================================================
y_pred_selected = rf_selected.predict(X_test_selected)
y_probs_selected = rf_selected.predict_proba(X_test_selected)[:, 1]

# Print Model Performance
print("\nModel Performance Using 19 Features:")
print(f"Test Accuracy: {accuracy_score(y_test, y_pred_selected):.4f}")
print(f"Balanced Accuracy: {balanced_accuracy_score(y_test, y_pred_selected):.4f}")
print(f"MCC: {matthews_corrcoef(y_test, y_pred_selected):.4f}")
print(f"Log Loss: {log_loss(y_test, y_probs_selected):.4f}")
print(f"F1 Score: {f1_score(y_test, y_pred_selected):.4f}")
print(f"Recall: {recall_score(y_test, y_pred_selected):.4f}")
print(f"Precision: {precision_score(y_test, y_pred_selected):.4f}")
print(classification_report(y_test, y_pred_selected, target_names=['CH1', 'CH2']))

# Compute Confusion Matrix, FPR, and FNR
cm = confusion_matrix(y_test, y_pred_selected)
print("\nConfusion Matrix:")
print(cm)
fp_rate = cm[0][1] / (cm[0][1] + cm[0][0])
fn_rate = cm[1][0] / (cm[1][0] + cm[1][1])
print(f"False Positive Rate (FPR): {fp_rate:.4f}")
print(f"False Negative Rate (FNR): {fn_rate:.4f}")


✅ Loaded Probe3 ADC3 CH1: (1998, 38)
✅ Loaded Probe3 ADC3 CH2: (1998, 38)

Model Performance Using 19 Features:
Test Accuracy: 1.0000
Balanced Accuracy: 1.0000
MCC: 1.0000
Log Loss: 0.0415
F1 Score: 1.0000
Recall: 1.0000
Precision: 1.0000
              precision    recall  f1-score   support

         CH1       1.00      1.00      1.00      1998
         CH2       1.00      1.00      1.00      1998

    accuracy                           1.00      3996
   macro avg       1.00      1.00      1.00      3996
weighted avg       1.00      1.00      1.00      3996


Confusion Matrix:
[[1998    0]
 [   0 1998]]
False Positive Rate (FPR): 0.0000
False Negative Rate (FNR): 0.0000


# 100 MSPS

## Training the model with 19 Features


In [18]:
import numpy as np
import os
import joblib
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import skew, kurtosis
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score, classification_report, confusion_matrix,
    roc_curve, auc, precision_recall_curve, matthews_corrcoef, 
    balanced_accuracy_score, log_loss, f1_score, recall_score, precision_score
)

# ==================================================================
# Fixed Configuration
# ==================================================================
base_directory = "C:\\Users\\awm21\\Documents\\Probe_Vari\\Neural_Networks\\Features_Vitis"

adc_num = 1
train_probe_dataset = 1
test_probe_dataset = 2
sample_rate = 100

train_probe = f"Probe{train_probe_dataset}"
test_probe = f"Probe{test_probe_dataset}"

# List of all 38 loaded features
all_feature_names = [
    "Max_H", "Min_H", "Mean_H", "Std_H", "Mean Deviation_H", "RMS_H", "Skewness_H", "Kurtosis_H", "Peak-to-Peak_H", "Zero Crossing Rate_H",
    "Centroid_H", "Entropy_H", "Spread_H", "Skewness_Freq_H", "Mean_Freq_H", "Kurtosis_Freq_H", "Irregularity_H", "Variance_H", "Dominant_Freq_H",
    "Max_L", "Min_L", "Mean_L", "Std_L", "Mean Deviation_L", "RMS_L", "Skewness_L", "Kurtosis_L", "Peak-to-Peak_L", "Zero Crossing Rate_L",
    "Centroid_L", "Entropy_L", "Spread_L", "Skewness_Freq_L", "Mean_Freq_L", "Kurtosis_Freq_L", "Irregularity_L", "Variance_L"
]

# Select only the 19 required features from the 38 loaded features
selected_features = [
    "Mean_H", "RMS_H", "Mean_L", "Variance_L", "RMS_L", "Mean_Freq_H", "Entropy_L", 
    "Max_H", "Entropy_H", "Mean Deviation_H", "Min_L", "Variance_H", "Mean_Freq_L", 
    "Min_H", "Mean Deviation_L", "Max_L", "Std_L", "Skewness_H", "Skewness_L"
]

# Get indices of selected features in the original dataset
selected_feature_indices = [all_feature_names.index(f) for f in selected_features]

def load_channel_data(probe_name, adc_num, channel):
    """Load specific ADC/channel data for a probe and verify ADC consistency."""
    folder = os.path.join(base_directory, f"P{probe_name[-1]}_fd")
    filename = f'fd_{probe_name}_ADC{adc_num}_CH{channel}_{sample_rate}_MSPS.npy'
    file_path = os.path.join(folder, filename)
    
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"File not found: {file_path}")
    
    data = np.load(file_path)
    print(f"✅ Loaded {probe_name} ADC{adc_num} CH{channel}: {data.shape}")
    return data

# ==================================================================
# Training & Test Data
# ==================================================================
train_ch0 = load_channel_data(train_probe, adc_num, 1)
train_ch1 = load_channel_data(train_probe, adc_num, 2)
test_ch0 = load_channel_data(test_probe, adc_num, 1)
test_ch1 = load_channel_data(test_probe, adc_num, 2)

# Stack training and test data
X_train = np.vstack([train_ch0, train_ch1])
y_train = np.concatenate([np.zeros(train_ch0.shape[0]), np.ones(train_ch1.shape[0])])
X_test = np.vstack([test_ch0, test_ch1])
y_test = np.concatenate([np.zeros(test_ch0.shape[0]), np.ones(test_ch1.shape[0])])

# ==================================================================
# Data Preprocessing
# ==================================================================
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Select only required 19 features from the 38 loaded dataset
X_train_selected = X_train_scaled[:, selected_feature_indices]
X_test_selected = X_test_scaled[:, selected_feature_indices]

# ==================================================================
# Train Model with Selected Features (19 Features)
# ==================================================================
rf = RandomForestClassifier(
    n_estimators=50,
    max_depth=10,
    min_samples_split=10,
    min_samples_leaf=5,
    max_features='sqrt',
    random_state=42
)
rf.fit(X_train_selected, y_train)

# Get feature importance only for selected features
feature_importances = rf.feature_importances_

# Print Selected Features and their Importances
print("\nSelected Features and Importances:")
for feature_idx, feature_name in zip(range(len(selected_features)), selected_features):
    print(f"{feature_name}: Importance {feature_importances[feature_idx]:.4f}")

# ==================================================================
# Retrain Model with Selected Features
# ==================================================================
rf_selected = RandomForestClassifier(
    n_estimators=50,
    max_depth=10,
    min_samples_split=10,
    min_samples_leaf=5,
    max_features='sqrt',
    random_state=42
)
rf_selected.fit(X_train_selected, y_train)

# Create a unique folder name dynamically
model_dir = f"Models_test/19_Feature_Tuning_{train_probe}_to_{test_probe}_ADC{adc_num}_{sample_rate}_MSPS"
os.makedirs(model_dir, exist_ok=True)

# Save the Model and Scaler
joblib.dump(rf_selected, os.path.join(model_dir, "RandomForest_Model.pkl"))
joblib.dump(scaler, os.path.join(model_dir, "Scaler.pkl"))
joblib.dump(selected_feature_indices, os.path.join(model_dir, "Selected_Features.pkl"))

# Predictions
y_pred_selected = rf_selected.predict(X_test_selected)
y_probs_selected = rf_selected.predict_proba(X_test_selected)[:, 1]

# ==================================================================
# Performance Metrics After Feature Selection
# ==================================================================
print("\nModel Performance Using 19 Features:")
print(f"Test Accuracy: {accuracy_score(y_test, y_pred_selected):.4f}")
print(f"Balanced Accuracy: {balanced_accuracy_score(y_test, y_pred_selected):.4f}")
print(f"MCC: {matthews_corrcoef(y_test, y_pred_selected):.4f}")
print(f"Log Loss: {log_loss(y_test, y_probs_selected):.4f}")
print(f"F1 Score: {f1_score(y_test, y_pred_selected):.4f}")
print(f"Recall: {recall_score(y_test, y_pred_selected):.4f}")
print(f"Precision: {precision_score(y_test, y_pred_selected):.4f}")
print(classification_report(y_test, y_pred_selected, target_names=['CH1', 'CH2']))

# Compute Confusion Matrix, FPR, and FNR
cm = confusion_matrix(y_test, y_pred_selected)
print("\nConfusion Matrix:")
print(cm)
fp_rate = cm[0][1] / (cm[0][1] + cm[0][0])
fn_rate = cm[1][0] / (cm[1][0] + cm[1][1])
print(f"False Positive Rate (FPR): {fp_rate:.4f}")
print(f"False Negative Rate (FNR): {fn_rate:.4f}")


✅ Loaded Probe1 ADC1 CH1: (998, 38)
✅ Loaded Probe1 ADC1 CH2: (998, 38)
✅ Loaded Probe2 ADC1 CH1: (998, 38)
✅ Loaded Probe2 ADC1 CH2: (998, 38)

Selected Features and Importances:
Mean_H: Importance 0.1886
RMS_H: Importance 0.1493
Mean_L: Importance 0.1782
Variance_L: Importance 0.0800
RMS_L: Importance 0.1576
Mean_Freq_H: Importance 0.0000
Entropy_L: Importance 0.0000
Max_H: Importance 0.0457
Entropy_H: Importance 0.0205
Mean Deviation_H: Importance 0.0230
Min_L: Importance 0.0094
Variance_H: Importance 0.1063
Mean_Freq_L: Importance 0.0000
Min_H: Importance 0.0000
Mean Deviation_L: Importance 0.0406
Max_L: Importance 0.0000
Std_L: Importance 0.0000
Skewness_H: Importance 0.0007
Skewness_L: Importance 0.0000

Model Performance Using 19 Features:
Test Accuracy: 1.0000
Balanced Accuracy: 1.0000
MCC: 1.0000
Log Loss: 0.0003
F1 Score: 1.0000
Recall: 1.0000
Precision: 1.0000
              precision    recall  f1-score   support

         CH1       1.00      1.00      1.00       998
       

## Testing on the models trained from above


### ADC2 Probe3 Test at 100MSPS

In [33]:
import numpy as np
import os
import joblib
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score, classification_report, confusion_matrix, log_loss,
    f1_score, recall_score, precision_score, matthews_corrcoef, balanced_accuracy_score
)

# ==================================================================
# Fixed Configuration
# ==================================================================
base_directory = "C:\\Users\\awm21\\Documents\\Probe_Vari\\Neural_Networks\\Features_Vitis"

adc_num = 2
test_probe_dataset = 3
sample_rate = 100

test_probe = f"Probe{test_probe_dataset}"

# Directory where the model and scaler were saved
model_dir = f"Models_test/19_Feature_Tuning_Probe1_to_Probe2_ADC1_100_MSPS"

# Load the trained model and scaler
model_path = os.path.join(model_dir, "RandomForest_Model.pkl")
scaler_path = os.path.join(model_dir, "Scaler.pkl")
features_path = os.path.join(model_dir, "Selected_Features.pkl")

if not os.path.exists(model_path) or not os.path.exists(scaler_path) or not os.path.exists(features_path):
    raise FileNotFoundError("Model, scaler, or selected features file not found.")

rf_selected = joblib.load(model_path)
scaler = joblib.load(scaler_path)
selected_feature_indices = joblib.load(features_path)

# List of all 38 loaded features
all_feature_names = [
    "Max_H", "Min_H", "Mean_H", "Std_H", "Mean Deviation_H", "RMS_H", "Skewness_H", "Kurtosis_H", "Peak-to-Peak_H", "Zero Crossing Rate_H",
    "Centroid_H", "Entropy_H", "Spread_H", "Skewness_Freq_H", "Mean_Freq_H", "Kurtosis_Freq_H", "Irregularity_H", "Variance_H", "Dominant_Freq_H",
    "Max_L", "Min_L", "Mean_L", "Std_L", "Mean Deviation_L", "RMS_L", "Skewness_L", "Kurtosis_L", "Peak-to-Peak_L", "Zero Crossing Rate_L",
    "Centroid_L", "Entropy_L", "Spread_L", "Skewness_Freq_L", "Mean_Freq_L", "Kurtosis_Freq_L", "Irregularity_L", "Variance_L"
]

# Get feature names for selected features
selected_features = [all_feature_names[i] for i in selected_feature_indices]

def load_channel_data(probe_name, adc_num, channel):
    """Load specific ADC/channel data for a probe and verify ADC consistency."""
    folder = os.path.join(base_directory, f"P{probe_name[-1]}_fd")
    filename = f'fd_{probe_name}_ADC{adc_num}_CH{channel}_{sample_rate}_MSPS.npy'
    file_path = os.path.join(folder, filename)
    
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"File not found: {file_path}")
    
    data = np.load(file_path)
    print(f"✅ Loaded {probe_name} ADC{adc_num} CH{channel}: {data.shape}")
    return data

# ==================================================================
# Load Test Data
# ==================================================================
test_ch0 = load_channel_data(test_probe, adc_num, 1)
test_ch1 = load_channel_data(test_probe, adc_num, 2)

# Stack test data
X_test = np.vstack([test_ch0, test_ch1])
y_test = np.concatenate([np.zeros(test_ch0.shape[0]), np.ones(test_ch1.shape[0])])

# Apply standardization using the loaded scaler
X_test_scaled = scaler.transform(X_test)

# Select only required 19 features from the 38 loaded dataset
X_test_selected = X_test_scaled[:, selected_feature_indices]

# ==================================================================
# Evaluate the Model
# ==================================================================
y_pred_selected = rf_selected.predict(X_test_selected)
y_probs_selected = rf_selected.predict_proba(X_test_selected)[:, 1]

# Print Model Performance
print("\nModel Performance Using 19 Features:")
print(f"Test Accuracy: {accuracy_score(y_test, y_pred_selected):.4f}")
print(f"Balanced Accuracy: {balanced_accuracy_score(y_test, y_pred_selected):.4f}")
print(f"MCC: {matthews_corrcoef(y_test, y_pred_selected):.4f}")
print(f"Log Loss: {log_loss(y_test, y_probs_selected):.4f}")
print(f"F1 Score: {f1_score(y_test, y_pred_selected):.4f}")
print(f"Recall: {recall_score(y_test, y_pred_selected):.4f}")
print(f"Precision: {precision_score(y_test, y_pred_selected):.4f}")
print(classification_report(y_test, y_pred_selected, target_names=['CH1', 'CH2']))

# Compute Confusion Matrix, FPR, and FNR
cm = confusion_matrix(y_test, y_pred_selected)
print("\nConfusion Matrix:")
print(cm)
fp_rate = cm[0][1] / (cm[0][1] + cm[0][0])
fn_rate = cm[1][0] / (cm[1][0] + cm[1][1])
print(f"False Positive Rate (FPR): {fp_rate:.4f}")
print(f"False Negative Rate (FNR): {fn_rate:.4f}")


✅ Loaded Probe3 ADC2 CH1: (998, 38)
✅ Loaded Probe3 ADC2 CH2: (998, 38)

Model Performance Using 19 Features:
Test Accuracy: 1.0000
Balanced Accuracy: 1.0000
MCC: 1.0000
Log Loss: 0.0015
F1 Score: 1.0000
Recall: 1.0000
Precision: 1.0000
              precision    recall  f1-score   support

         CH1       1.00      1.00      1.00       998
         CH2       1.00      1.00      1.00       998

    accuracy                           1.00      1996
   macro avg       1.00      1.00      1.00      1996
weighted avg       1.00      1.00      1.00      1996


Confusion Matrix:
[[998   0]
 [  0 998]]
False Positive Rate (FPR): 0.0000
False Negative Rate (FNR): 0.0000


### ADC3 Probe3 Test at 100MSPS

In [32]:
import numpy as np
import os
import joblib
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score, classification_report, confusion_matrix, log_loss,
    f1_score, recall_score, precision_score, matthews_corrcoef, balanced_accuracy_score
)

# ==================================================================
# Fixed Configuration
# ==================================================================
base_directory = "C:\\Users\\awm21\\Documents\\Probe_Vari\\Neural_Networks\\Features_Vitis"

adc_num = 3
test_probe_dataset = 3
sample_rate = 100

test_probe = f"Probe{test_probe_dataset}"

# Directory where the model and scaler were saved
model_dir = f"Models_test/19_Feature_Tuning_Probe1_to_Probe2_ADC1_100_MSPS"

# Load the trained model and scaler
model_path = os.path.join(model_dir, "RandomForest_Model.pkl")
scaler_path = os.path.join(model_dir, "Scaler.pkl")
features_path = os.path.join(model_dir, "Selected_Features.pkl")

if not os.path.exists(model_path) or not os.path.exists(scaler_path) or not os.path.exists(features_path):
    raise FileNotFoundError("Model, scaler, or selected features file not found.")

rf_selected = joblib.load(model_path)
scaler = joblib.load(scaler_path)
selected_feature_indices = joblib.load(features_path)

# List of all 38 loaded features
all_feature_names = [
    "Max_H", "Min_H", "Mean_H", "Std_H", "Mean Deviation_H", "RMS_H", "Skewness_H", "Kurtosis_H", "Peak-to-Peak_H", "Zero Crossing Rate_H",
    "Centroid_H", "Entropy_H", "Spread_H", "Skewness_Freq_H", "Mean_Freq_H", "Kurtosis_Freq_H", "Irregularity_H", "Variance_H", "Dominant_Freq_H",
    "Max_L", "Min_L", "Mean_L", "Std_L", "Mean Deviation_L", "RMS_L", "Skewness_L", "Kurtosis_L", "Peak-to-Peak_L", "Zero Crossing Rate_L",
    "Centroid_L", "Entropy_L", "Spread_L", "Skewness_Freq_L", "Mean_Freq_L", "Kurtosis_Freq_L", "Irregularity_L", "Variance_L"
]

# Get feature names for selected features
selected_features = [all_feature_names[i] for i in selected_feature_indices]

def load_channel_data(probe_name, adc_num, channel):
    """Load specific ADC/channel data for a probe and verify ADC consistency."""
    folder = os.path.join(base_directory, f"P{probe_name[-1]}_fd")
    filename = f'fd_{probe_name}_ADC{adc_num}_CH{channel}_{sample_rate}_MSPS.npy'
    file_path = os.path.join(folder, filename)
    
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"File not found: {file_path}")
    
    data = np.load(file_path)
    print(f"✅ Loaded {probe_name} ADC{adc_num} CH{channel}: {data.shape}")
    return data

# ==================================================================
# Load Test Data
# ==================================================================
test_ch0 = load_channel_data(test_probe, adc_num, 1)
test_ch1 = load_channel_data(test_probe, adc_num, 2)

# Stack test data
X_test = np.vstack([test_ch0, test_ch1])
y_test = np.concatenate([np.zeros(test_ch0.shape[0]), np.ones(test_ch1.shape[0])])

# Apply standardization using the loaded scaler
X_test_scaled = scaler.transform(X_test)

# Select only required 19 features from the 38 loaded dataset
X_test_selected = X_test_scaled[:, selected_feature_indices]

# ==================================================================
# Evaluate the Model
# ==================================================================
y_pred_selected = rf_selected.predict(X_test_selected)
y_probs_selected = rf_selected.predict_proba(X_test_selected)[:, 1]

# Print Model Performance
print("\nModel Performance Using 19 Features:")
print(f"Test Accuracy: {accuracy_score(y_test, y_pred_selected):.4f}")
print(f"Balanced Accuracy: {balanced_accuracy_score(y_test, y_pred_selected):.4f}")
print(f"MCC: {matthews_corrcoef(y_test, y_pred_selected):.4f}")
print(f"Log Loss: {log_loss(y_test, y_probs_selected):.4f}")
print(f"F1 Score: {f1_score(y_test, y_pred_selected):.4f}")
print(f"Recall: {recall_score(y_test, y_pred_selected):.4f}")
print(f"Precision: {precision_score(y_test, y_pred_selected):.4f}")
print(classification_report(y_test, y_pred_selected, target_names=['CH1', 'CH2']))

# Compute Confusion Matrix, FPR, and FNR
cm = confusion_matrix(y_test, y_pred_selected)
print("\nConfusion Matrix:")
print(cm)
fp_rate = cm[0][1] / (cm[0][1] + cm[0][0])
fn_rate = cm[1][0] / (cm[1][0] + cm[1][1])
print(f"False Positive Rate (FPR): {fp_rate:.4f}")
print(f"False Negative Rate (FNR): {fn_rate:.4f}")


✅ Loaded Probe3 ADC3 CH1: (998, 38)
✅ Loaded Probe3 ADC3 CH2: (998, 38)

Model Performance Using 19 Features:
Test Accuracy: 1.0000
Balanced Accuracy: 1.0000
MCC: 1.0000
Log Loss: 0.0016
F1 Score: 1.0000
Recall: 1.0000
Precision: 1.0000
              precision    recall  f1-score   support

         CH1       1.00      1.00      1.00       998
         CH2       1.00      1.00      1.00       998

    accuracy                           1.00      1996
   macro avg       1.00      1.00      1.00      1996
weighted avg       1.00      1.00      1.00      1996


Confusion Matrix:
[[998   0]
 [  0 998]]
False Positive Rate (FPR): 0.0000
False Negative Rate (FNR): 0.0000


# 150 MSPS 

## Training the model with 19 Features


In [21]:

import numpy as np
import os
import joblib
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import skew, kurtosis
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score, classification_report, confusion_matrix,
    roc_curve, auc, precision_recall_curve, matthews_corrcoef, 
    balanced_accuracy_score, log_loss, f1_score, recall_score, precision_score
)

# ==================================================================
# Fixed Configuration
# ==================================================================
base_directory = "C:\\Users\\awm21\\Documents\\Probe_Vari\\Neural_Networks\\Features_Vitis"

adc_num = 1
train_probe_dataset = 1
test_probe_dataset = 2
sample_rate = 150

train_probe = f"Probe{train_probe_dataset}"
test_probe = f"Probe{test_probe_dataset}"

# List of all 38 loaded features
all_feature_names = [
    "Max_H", "Min_H", "Mean_H", "Std_H", "Mean Deviation_H", "RMS_H", "Skewness_H", "Kurtosis_H", "Peak-to-Peak_H", "Zero Crossing Rate_H",
    "Centroid_H", "Entropy_H", "Spread_H", "Skewness_Freq_H", "Mean_Freq_H", "Kurtosis_Freq_H", "Irregularity_H", "Variance_H", "Dominant_Freq_H",
    "Max_L", "Min_L", "Mean_L", "Std_L", "Mean Deviation_L", "RMS_L", "Skewness_L", "Kurtosis_L", "Peak-to-Peak_L", "Zero Crossing Rate_L",
    "Centroid_L", "Entropy_L", "Spread_L", "Skewness_Freq_L", "Mean_Freq_L", "Kurtosis_Freq_L", "Irregularity_L", "Variance_L"
]

# Select only the 19 required features from the 38 loaded features
selected_features = [
    "Mean_H", "RMS_H", "Mean_L", "Variance_L", "RMS_L", "Mean_Freq_H", "Entropy_L", 
    "Max_H", "Entropy_H", "Mean Deviation_H", "Min_L", "Variance_H", "Mean_Freq_L", 
    "Min_H", "Mean Deviation_L", "Max_L", "Std_L", "Skewness_H", "Skewness_L"
]

# Get indices of selected features in the original dataset
selected_feature_indices = [all_feature_names.index(f) for f in selected_features]

def load_channel_data(probe_name, adc_num, channel):
    """Load specific ADC/channel data for a probe and verify ADC consistency."""
    folder = os.path.join(base_directory, f"P{probe_name[-1]}_fd")
    filename = f'fd_{probe_name}_ADC{adc_num}_CH{channel}_{sample_rate}_MSPS.npy'
    file_path = os.path.join(folder, filename)
    
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"File not found: {file_path}")
    
    data = np.load(file_path)
    print(f"✅ Loaded {probe_name} ADC{adc_num} CH{channel}: {data.shape}")
    return data

# ==================================================================
# Training & Test Data
# ==================================================================
train_ch0 = load_channel_data(train_probe, adc_num, 1)
train_ch1 = load_channel_data(train_probe, adc_num, 2)
test_ch0 = load_channel_data(test_probe, adc_num, 1)
test_ch1 = load_channel_data(test_probe, adc_num, 2)

# Stack training and test data
X_train = np.vstack([train_ch0, train_ch1])
y_train = np.concatenate([np.zeros(train_ch0.shape[0]), np.ones(train_ch1.shape[0])])
X_test = np.vstack([test_ch0, test_ch1])
y_test = np.concatenate([np.zeros(test_ch0.shape[0]), np.ones(test_ch1.shape[0])])

# ==================================================================
# Data Preprocessing
# ==================================================================
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Select only required 19 features from the 38 loaded dataset
X_train_selected = X_train_scaled[:, selected_feature_indices]
X_test_selected = X_test_scaled[:, selected_feature_indices]

# ==================================================================
# Train Model with Selected Features (19 Features)
# ==================================================================
rf = RandomForestClassifier(
    n_estimators=50,
    max_depth=10,
    min_samples_split=10,
    min_samples_leaf=5,
    max_features='sqrt',
    random_state=42
)
rf.fit(X_train_selected, y_train)

# Get feature importance only for selected features
feature_importances = rf.feature_importances_

# Print Selected Features and their Importances
print("\nSelected Features and Importances:")
for feature_idx, feature_name in zip(range(len(selected_features)), selected_features):
    print(f"{feature_name}: Importance {feature_importances[feature_idx]:.4f}")

# ==================================================================
# Retrain Model with Selected Features
# ==================================================================
rf_selected = RandomForestClassifier(
    n_estimators=50,
    max_depth=10,
    min_samples_split=10,
    min_samples_leaf=5,
    max_features='sqrt',
    random_state=42
)
rf_selected.fit(X_train_selected, y_train)

# Create a unique folder name dynamically
model_dir = f"Models_test/19_Feature_Tuning_{train_probe}_to_{test_probe}_ADC{adc_num}_{sample_rate}_MSPS"
os.makedirs(model_dir, exist_ok=True)

# Save the Model and Scaler
joblib.dump(rf_selected, os.path.join(model_dir, "RandomForest_Model.pkl"))
joblib.dump(scaler, os.path.join(model_dir, "Scaler.pkl"))
joblib.dump(selected_feature_indices, os.path.join(model_dir, "Selected_Features.pkl"))

# Predictions
y_pred_selected = rf_selected.predict(X_test_selected)
y_probs_selected = rf_selected.predict_proba(X_test_selected)[:, 1]

# ==================================================================
# Performance Metrics After Feature Selection
# ==================================================================
print("\nModel Performance Using 19 Features:")
print(f"Test Accuracy: {accuracy_score(y_test, y_pred_selected):.4f}")
print(f"Balanced Accuracy: {balanced_accuracy_score(y_test, y_pred_selected):.4f}")
print(f"MCC: {matthews_corrcoef(y_test, y_pred_selected):.4f}")
print(f"Log Loss: {log_loss(y_test, y_probs_selected):.4f}")
print(f"F1 Score: {f1_score(y_test, y_pred_selected):.4f}")
print(f"Recall: {recall_score(y_test, y_pred_selected):.4f}")
print(f"Precision: {precision_score(y_test, y_pred_selected):.4f}")
print(classification_report(y_test, y_pred_selected, target_names=['CH1', 'CH2']))

# Compute Confusion Matrix, FPR, and FNR
cm = confusion_matrix(y_test, y_pred_selected)
print("\nConfusion Matrix:")
print(cm)
fp_rate = cm[0][1] / (cm[0][1] + cm[0][0])
fn_rate = cm[1][0] / (cm[1][0] + cm[1][1])
print(f"False Positive Rate (FPR): {fp_rate:.4f}")
print(f"False Negative Rate (FNR): {fn_rate:.4f}")


✅ Loaded Probe1 ADC1 CH1: (665, 38)
✅ Loaded Probe1 ADC1 CH2: (665, 38)
✅ Loaded Probe2 ADC1 CH1: (665, 38)
✅ Loaded Probe2 ADC1 CH2: (665, 38)

Selected Features and Importances:
Mean_H: Importance 0.1200
RMS_H: Importance 0.1208
Mean_L: Importance 0.1000
Variance_L: Importance 0.0800
RMS_L: Importance 0.0610
Mean_Freq_H: Importance 0.0190
Entropy_L: Importance 0.0000
Max_H: Importance 0.1000
Entropy_H: Importance 0.0382
Mean Deviation_H: Importance 0.1200
Min_L: Importance 0.0010
Variance_H: Importance 0.1000
Mean_Freq_L: Importance 0.0000
Min_H: Importance 0.0000
Mean Deviation_L: Importance 0.1400
Max_L: Importance 0.0000
Std_L: Importance 0.0000
Skewness_H: Importance 0.0000
Skewness_L: Importance 0.0000

Model Performance Using 19 Features:
Test Accuracy: 1.0000
Balanced Accuracy: 1.0000
MCC: 1.0000
Log Loss: 0.0004
F1 Score: 1.0000
Recall: 1.0000
Precision: 1.0000
              precision    recall  f1-score   support

         CH1       1.00      1.00      1.00       665
       

## Testing on the models trained from above


### ADC2 Probe3 Test at 150MSPS

In [22]:

import numpy as np
import os
import joblib
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score, classification_report, confusion_matrix, log_loss,
    f1_score, recall_score, precision_score, matthews_corrcoef, balanced_accuracy_score
)

# ==================================================================
# Fixed Configuration
# ==================================================================
base_directory = "C:\\Users\\awm21\\Documents\\Probe_Vari\\Neural_Networks\\Features_Vitis"

adc_num = 2
test_probe_dataset = 3
sample_rate = 150

test_probe = f"Probe{test_probe_dataset}"

# Directory where the model and scaler were saved
model_dir = f"Models_test/19_Feature_Tuning_Probe1_to_Probe2_ADC1_150_MSPS"

# Load the trained model and scaler
model_path = os.path.join(model_dir, "RandomForest_Model.pkl")
scaler_path = os.path.join(model_dir, "Scaler.pkl")
features_path = os.path.join(model_dir, "Selected_Features.pkl")

if not os.path.exists(model_path) or not os.path.exists(scaler_path) or not os.path.exists(features_path):
    raise FileNotFoundError("Model, scaler, or selected features file not found.")

rf_selected = joblib.load(model_path)
scaler = joblib.load(scaler_path)
selected_feature_indices = joblib.load(features_path)

# List of all 38 loaded features
all_feature_names = [
    "Max_H", "Min_H", "Mean_H", "Std_H", "Mean Deviation_H", "RMS_H", "Skewness_H", "Kurtosis_H", "Peak-to-Peak_H", "Zero Crossing Rate_H",
    "Centroid_H", "Entropy_H", "Spread_H", "Skewness_Freq_H", "Mean_Freq_H", "Kurtosis_Freq_H", "Irregularity_H", "Variance_H", "Dominant_Freq_H",
    "Max_L", "Min_L", "Mean_L", "Std_L", "Mean Deviation_L", "RMS_L", "Skewness_L", "Kurtosis_L", "Peak-to-Peak_L", "Zero Crossing Rate_L",
    "Centroid_L", "Entropy_L", "Spread_L", "Skewness_Freq_L", "Mean_Freq_L", "Kurtosis_Freq_L", "Irregularity_L", "Variance_L"
]

# Get feature names for selected features
selected_features = [all_feature_names[i] for i in selected_feature_indices]

def load_channel_data(probe_name, adc_num, channel):
    """Load specific ADC/channel data for a probe and verify ADC consistency."""
    folder = os.path.join(base_directory, f"P{probe_name[-1]}_fd")
    filename = f'fd_{probe_name}_ADC{adc_num}_CH{channel}_{sample_rate}_MSPS.npy'
    file_path = os.path.join(folder, filename)
    
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"File not found: {file_path}")
    
    data = np.load(file_path)
    print(f"✅ Loaded {probe_name} ADC{adc_num} CH{channel}: {data.shape}")
    return data

# ==================================================================
# Load Test Data
# ==================================================================
test_ch0 = load_channel_data(test_probe, adc_num, 1)
test_ch1 = load_channel_data(test_probe, adc_num, 2)

# Stack test data
X_test = np.vstack([test_ch0, test_ch1])
y_test = np.concatenate([np.zeros(test_ch0.shape[0]), np.ones(test_ch1.shape[0])])

# Apply standardization using the loaded scaler
X_test_scaled = scaler.transform(X_test)

# Select only required 19 features from the 38 loaded dataset
X_test_selected = X_test_scaled[:, selected_feature_indices]

# ==================================================================
# Evaluate the Model
# ==================================================================
y_pred_selected = rf_selected.predict(X_test_selected)
y_probs_selected = rf_selected.predict_proba(X_test_selected)[:, 1]

# Print Model Performance
print("\nModel Performance Using 19 Features:")
print(f"Test Accuracy: {accuracy_score(y_test, y_pred_selected):.4f}")
print(f"Balanced Accuracy: {balanced_accuracy_score(y_test, y_pred_selected):.4f}")
print(f"MCC: {matthews_corrcoef(y_test, y_pred_selected):.4f}")
print(f"Log Loss: {log_loss(y_test, y_probs_selected):.4f}")
print(f"F1 Score: {f1_score(y_test, y_pred_selected):.4f}")
print(f"Recall: {recall_score(y_test, y_pred_selected):.4f}")
print(f"Precision: {precision_score(y_test, y_pred_selected):.4f}")
print(classification_report(y_test, y_pred_selected, target_names=['CH1', 'CH2']))

# Compute Confusion Matrix, FPR, and FNR
cm = confusion_matrix(y_test, y_pred_selected)
print("\nConfusion Matrix:")
print(cm)
fp_rate = cm[0][1] / (cm[0][1] + cm[0][0])
fn_rate = cm[1][0] / (cm[1][0] + cm[1][1])
print(f"False Positive Rate (FPR): {fp_rate:.4f}")
print(f"False Negative Rate (FNR): {fn_rate:.4f}")


✅ Loaded Probe3 ADC2 CH1: (665, 38)
✅ Loaded Probe3 ADC2 CH2: (665, 38)

Model Performance Using 19 Features:
Test Accuracy: 1.0000
Balanced Accuracy: 1.0000
MCC: 1.0000
Log Loss: 0.0462
F1 Score: 1.0000
Recall: 1.0000
Precision: 1.0000
              precision    recall  f1-score   support

         CH1       1.00      1.00      1.00       665
         CH2       1.00      1.00      1.00       665

    accuracy                           1.00      1330
   macro avg       1.00      1.00      1.00      1330
weighted avg       1.00      1.00      1.00      1330


Confusion Matrix:
[[665   0]
 [  0 665]]
False Positive Rate (FPR): 0.0000
False Negative Rate (FNR): 0.0000


### ADC3 Probe3 Test at 150MSPS

In [23]:

import numpy as np
import os
import joblib
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score, classification_report, confusion_matrix, log_loss,
    f1_score, recall_score, precision_score, matthews_corrcoef, balanced_accuracy_score
)

# ==================================================================
# Fixed Configuration
# ==================================================================
base_directory = "C:\\Users\\awm21\\Documents\\Probe_Vari\\Neural_Networks\\Features_Vitis"

adc_num = 3
test_probe_dataset = 3
sample_rate = 150

test_probe = f"Probe{test_probe_dataset}"

# Directory where the model and scaler were saved
model_dir = f"Models_test/19_Feature_Tuning_Probe1_to_Probe2_ADC1_150_MSPS"

# Load the trained model and scaler
model_path = os.path.join(model_dir, "RandomForest_Model.pkl")
scaler_path = os.path.join(model_dir, "Scaler.pkl")
features_path = os.path.join(model_dir, "Selected_Features.pkl")

if not os.path.exists(model_path) or not os.path.exists(scaler_path) or not os.path.exists(features_path):
    raise FileNotFoundError("Model, scaler, or selected features file not found.")

rf_selected = joblib.load(model_path)
scaler = joblib.load(scaler_path)
selected_feature_indices = joblib.load(features_path)

# List of all 38 loaded features
all_feature_names = [
    "Max_H", "Min_H", "Mean_H", "Std_H", "Mean Deviation_H", "RMS_H", "Skewness_H", "Kurtosis_H", "Peak-to-Peak_H", "Zero Crossing Rate_H",
    "Centroid_H", "Entropy_H", "Spread_H", "Skewness_Freq_H", "Mean_Freq_H", "Kurtosis_Freq_H", "Irregularity_H", "Variance_H", "Dominant_Freq_H",
    "Max_L", "Min_L", "Mean_L", "Std_L", "Mean Deviation_L", "RMS_L", "Skewness_L", "Kurtosis_L", "Peak-to-Peak_L", "Zero Crossing Rate_L",
    "Centroid_L", "Entropy_L", "Spread_L", "Skewness_Freq_L", "Mean_Freq_L", "Kurtosis_Freq_L", "Irregularity_L", "Variance_L"
]

# Get feature names for selected features
selected_features = [all_feature_names[i] for i in selected_feature_indices]

def load_channel_data(probe_name, adc_num, channel):
    """Load specific ADC/channel data for a probe and verify ADC consistency."""
    folder = os.path.join(base_directory, f"P{probe_name[-1]}_fd")
    filename = f'fd_{probe_name}_ADC{adc_num}_CH{channel}_{sample_rate}_MSPS.npy'
    file_path = os.path.join(folder, filename)
    
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"File not found: {file_path}")
    
    data = np.load(file_path)
    print(f"✅ Loaded {probe_name} ADC{adc_num} CH{channel}: {data.shape}")
    return data

# ==================================================================
# Load Test Data
# ==================================================================
test_ch0 = load_channel_data(test_probe, adc_num, 1)
test_ch1 = load_channel_data(test_probe, adc_num, 2)

# Stack test data
X_test = np.vstack([test_ch0, test_ch1])
y_test = np.concatenate([np.zeros(test_ch0.shape[0]), np.ones(test_ch1.shape[0])])

# Apply standardization using the loaded scaler
X_test_scaled = scaler.transform(X_test)

# Select only required 19 features from the 38 loaded dataset
X_test_selected = X_test_scaled[:, selected_feature_indices]

# ==================================================================
# Evaluate the Model
# ==================================================================
y_pred_selected = rf_selected.predict(X_test_selected)
y_probs_selected = rf_selected.predict_proba(X_test_selected)[:, 1]

# Print Model Performance
print("\nModel Performance Using 19 Features:")
print(f"Test Accuracy: {accuracy_score(y_test, y_pred_selected):.4f}")
print(f"Balanced Accuracy: {balanced_accuracy_score(y_test, y_pred_selected):.4f}")
print(f"MCC: {matthews_corrcoef(y_test, y_pred_selected):.4f}")
print(f"Log Loss: {log_loss(y_test, y_probs_selected):.4f}")
print(f"F1 Score: {f1_score(y_test, y_pred_selected):.4f}")
print(f"Recall: {recall_score(y_test, y_pred_selected):.4f}")
print(f"Precision: {precision_score(y_test, y_pred_selected):.4f}")
print(classification_report(y_test, y_pred_selected, target_names=['CH1', 'CH2']))

# Compute Confusion Matrix, FPR, and FNR
cm = confusion_matrix(y_test, y_pred_selected)
print("\nConfusion Matrix:")
print(cm)
fp_rate = cm[0][1] / (cm[0][1] + cm[0][0])
fn_rate = cm[1][0] / (cm[1][0] + cm[1][1])
print(f"False Positive Rate (FPR): {fp_rate:.4f}")
print(f"False Negative Rate (FNR): {fn_rate:.4f}")


✅ Loaded Probe3 ADC3 CH1: (665, 38)
✅ Loaded Probe3 ADC3 CH2: (665, 38)

Model Performance Using 19 Features:
Test Accuracy: 1.0000
Balanced Accuracy: 1.0000
MCC: 1.0000
Log Loss: 0.0549
F1 Score: 1.0000
Recall: 1.0000
Precision: 1.0000
              precision    recall  f1-score   support

         CH1       1.00      1.00      1.00       665
         CH2       1.00      1.00      1.00       665

    accuracy                           1.00      1330
   macro avg       1.00      1.00      1.00      1330
weighted avg       1.00      1.00      1.00      1330


Confusion Matrix:
[[665   0]
 [  0 665]]
False Positive Rate (FPR): 0.0000
False Negative Rate (FNR): 0.0000


# 200 MSPS

## Training the model with 19 Features


In [24]:

import numpy as np
import os
import joblib
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import skew, kurtosis
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score, classification_report, confusion_matrix,
    roc_curve, auc, precision_recall_curve, matthews_corrcoef, 
    balanced_accuracy_score, log_loss, f1_score, recall_score, precision_score
)

# ==================================================================
# Fixed Configuration
# ==================================================================
base_directory = "C:\\Users\\awm21\\Documents\\Probe_Vari\\Neural_Networks\\Features_Vitis"

adc_num = 1
train_probe_dataset = 1
test_probe_dataset = 2
sample_rate = 200

train_probe = f"Probe{train_probe_dataset}"
test_probe = f"Probe{test_probe_dataset}"

# List of all 38 loaded features
all_feature_names = [
    "Max_H", "Min_H", "Mean_H", "Std_H", "Mean Deviation_H", "RMS_H", "Skewness_H", "Kurtosis_H", "Peak-to-Peak_H", "Zero Crossing Rate_H",
    "Centroid_H", "Entropy_H", "Spread_H", "Skewness_Freq_H", "Mean_Freq_H", "Kurtosis_Freq_H", "Irregularity_H", "Variance_H", "Dominant_Freq_H",
    "Max_L", "Min_L", "Mean_L", "Std_L", "Mean Deviation_L", "RMS_L", "Skewness_L", "Kurtosis_L", "Peak-to-Peak_L", "Zero Crossing Rate_L",
    "Centroid_L", "Entropy_L", "Spread_L", "Skewness_Freq_L", "Mean_Freq_L", "Kurtosis_Freq_L", "Irregularity_L", "Variance_L"
]

# Select only the 19 required features from the 38 loaded features
selected_features = [
    "Mean_H", "RMS_H", "Mean_L", "Variance_L", "RMS_L", "Mean_Freq_H", "Entropy_L", 
    "Max_H", "Entropy_H", "Mean Deviation_H", "Min_L", "Variance_H", "Mean_Freq_L", 
    "Min_H", "Mean Deviation_L", "Max_L", "Std_L", "Skewness_H", "Skewness_L"
]

# Get indices of selected features in the original dataset
selected_feature_indices = [all_feature_names.index(f) for f in selected_features]

def load_channel_data(probe_name, adc_num, channel):
    """Load specific ADC/channel data for a probe and verify ADC consistency."""
    folder = os.path.join(base_directory, f"P{probe_name[-1]}_fd")
    filename = f'fd_{probe_name}_ADC{adc_num}_CH{channel}_{sample_rate}_MSPS.npy'
    file_path = os.path.join(folder, filename)
    
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"File not found: {file_path}")
    
    data = np.load(file_path)
    print(f"✅ Loaded {probe_name} ADC{adc_num} CH{channel}: {data.shape}")
    return data

# ==================================================================
# Training & Test Data
# ==================================================================
train_ch0 = load_channel_data(train_probe, adc_num, 1)
train_ch1 = load_channel_data(train_probe, adc_num, 2)
test_ch0 = load_channel_data(test_probe, adc_num, 1)
test_ch1 = load_channel_data(test_probe, adc_num, 2)

# Stack training and test data
X_train = np.vstack([train_ch0, train_ch1])
y_train = np.concatenate([np.zeros(train_ch0.shape[0]), np.ones(train_ch1.shape[0])])
X_test = np.vstack([test_ch0, test_ch1])
y_test = np.concatenate([np.zeros(test_ch0.shape[0]), np.ones(test_ch1.shape[0])])

# ==================================================================
# Data Preprocessing
# ==================================================================
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Select only required 19 features from the 38 loaded dataset
X_train_selected = X_train_scaled[:, selected_feature_indices]
X_test_selected = X_test_scaled[:, selected_feature_indices]

# ==================================================================
# Train Model with Selected Features (19 Features)
# ==================================================================
rf = RandomForestClassifier(
    n_estimators=50,
    max_depth=10,
    min_samples_split=10,
    min_samples_leaf=5,
    max_features='sqrt',
    random_state=42
)
rf.fit(X_train_selected, y_train)

# Get feature importance only for selected features
feature_importances = rf.feature_importances_

# Print Selected Features and their Importances
print("\nSelected Features and Importances:")
for feature_idx, feature_name in zip(range(len(selected_features)), selected_features):
    print(f"{feature_name}: Importance {feature_importances[feature_idx]:.4f}")

# ==================================================================
# Retrain Model with Selected Features
# ==================================================================
rf_selected = RandomForestClassifier(
    n_estimators=50,
    max_depth=10,
    min_samples_split=10,
    min_samples_leaf=5,
    max_features='sqrt',
    random_state=42
)
rf_selected.fit(X_train_selected, y_train)

# Create a unique folder name dynamically
model_dir = f"Models_test/19_Feature_Tuning_{train_probe}_to_{test_probe}_ADC{adc_num}_{sample_rate}_MSPS"
os.makedirs(model_dir, exist_ok=True)

# Save the Model and Scaler
joblib.dump(rf_selected, os.path.join(model_dir, "RandomForest_Model.pkl"))
joblib.dump(scaler, os.path.join(model_dir, "Scaler.pkl"))
joblib.dump(selected_feature_indices, os.path.join(model_dir, "Selected_Features.pkl"))

# Predictions
y_pred_selected = rf_selected.predict(X_test_selected)
y_probs_selected = rf_selected.predict_proba(X_test_selected)[:, 1]

# ==================================================================
# Performance Metrics After Feature Selection
# ==================================================================
print("\nModel Performance Using 19 Features:")
print(f"Test Accuracy: {accuracy_score(y_test, y_pred_selected):.4f}")
print(f"Balanced Accuracy: {balanced_accuracy_score(y_test, y_pred_selected):.4f}")
print(f"MCC: {matthews_corrcoef(y_test, y_pred_selected):.4f}")
print(f"Log Loss: {log_loss(y_test, y_probs_selected):.4f}")
print(f"F1 Score: {f1_score(y_test, y_pred_selected):.4f}")
print(f"Recall: {recall_score(y_test, y_pred_selected):.4f}")
print(f"Precision: {precision_score(y_test, y_pred_selected):.4f}")
print(classification_report(y_test, y_pred_selected, target_names=['CH1', 'CH2']))

# Compute Confusion Matrix, FPR, and FNR
cm = confusion_matrix(y_test, y_pred_selected)
print("\nConfusion Matrix:")
print(cm)
fp_rate = cm[0][1] / (cm[0][1] + cm[0][0])
fn_rate = cm[1][0] / (cm[1][0] + cm[1][1])
print(f"False Positive Rate (FPR): {fp_rate:.4f}")
print(f"False Negative Rate (FNR): {fn_rate:.4f}")


✅ Loaded Probe1 ADC1 CH1: (498, 38)
✅ Loaded Probe1 ADC1 CH2: (498, 38)
✅ Loaded Probe2 ADC1 CH1: (498, 38)
✅ Loaded Probe2 ADC1 CH2: (498, 38)

Selected Features and Importances:
Mean_H: Importance 0.1200
RMS_H: Importance 0.1202
Mean_L: Importance 0.1000
Variance_L: Importance 0.0800
RMS_L: Importance 0.0400
Mean_Freq_H: Importance 0.0000
Entropy_L: Importance 0.0195
Max_H: Importance 0.1000
Entropy_H: Importance 0.0000
Mean Deviation_H: Importance 0.1203
Min_L: Importance 0.0600
Variance_H: Importance 0.1000
Mean_Freq_L: Importance 0.0000
Min_H: Importance 0.0000
Mean Deviation_L: Importance 0.1400
Max_L: Importance 0.0000
Std_L: Importance 0.0000
Skewness_H: Importance 0.0000
Skewness_L: Importance 0.0000

Model Performance Using 19 Features:
Test Accuracy: 1.0000
Balanced Accuracy: 1.0000
MCC: 1.0000
Log Loss: 0.0082
F1 Score: 1.0000
Recall: 1.0000
Precision: 1.0000
              precision    recall  f1-score   support

         CH1       1.00      1.00      1.00       498
       

## Testing on the models trained from above


### ADC2 Probe3 Test at 200 MSPS

In [25]:
import numpy as np
import os
import joblib
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score, classification_report, confusion_matrix, log_loss,
    f1_score, recall_score, precision_score, matthews_corrcoef, balanced_accuracy_score
)

# ==================================================================
# Fixed Configuration
# ==================================================================
base_directory = "C:\\Users\\awm21\\Documents\\Probe_Vari\\Neural_Networks\\Features_Vitis"

adc_num = 2
test_probe_dataset = 3
sample_rate = 200

test_probe = f"Probe{test_probe_dataset}"

# Directory where the model and scaler were saved
model_dir = f"Models_test/19_Feature_Tuning_Probe1_to_Probe2_ADC1_200_MSPS"

# Load the trained model and scaler
model_path = os.path.join(model_dir, "RandomForest_Model.pkl")
scaler_path = os.path.join(model_dir, "Scaler.pkl")
features_path = os.path.join(model_dir, "Selected_Features.pkl")

if not os.path.exists(model_path) or not os.path.exists(scaler_path) or not os.path.exists(features_path):
    raise FileNotFoundError("Model, scaler, or selected features file not found.")

rf_selected = joblib.load(model_path)
scaler = joblib.load(scaler_path)
selected_feature_indices = joblib.load(features_path)

# List of all 38 loaded features
all_feature_names = [
    "Max_H", "Min_H", "Mean_H", "Std_H", "Mean Deviation_H", "RMS_H", "Skewness_H", "Kurtosis_H", "Peak-to-Peak_H", "Zero Crossing Rate_H",
    "Centroid_H", "Entropy_H", "Spread_H", "Skewness_Freq_H", "Mean_Freq_H", "Kurtosis_Freq_H", "Irregularity_H", "Variance_H", "Dominant_Freq_H",
    "Max_L", "Min_L", "Mean_L", "Std_L", "Mean Deviation_L", "RMS_L", "Skewness_L", "Kurtosis_L", "Peak-to-Peak_L", "Zero Crossing Rate_L",
    "Centroid_L", "Entropy_L", "Spread_L", "Skewness_Freq_L", "Mean_Freq_L", "Kurtosis_Freq_L", "Irregularity_L", "Variance_L"
]

# Get feature names for selected features
selected_features = [all_feature_names[i] for i in selected_feature_indices]

def load_channel_data(probe_name, adc_num, channel):
    """Load specific ADC/channel data for a probe and verify ADC consistency."""
    folder = os.path.join(base_directory, f"P{probe_name[-1]}_fd")
    filename = f'fd_{probe_name}_ADC{adc_num}_CH{channel}_{sample_rate}_MSPS.npy'
    file_path = os.path.join(folder, filename)
    
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"File not found: {file_path}")
    
    data = np.load(file_path)
    print(f"✅ Loaded {probe_name} ADC{adc_num} CH{channel}: {data.shape}")
    return data

# ==================================================================
# Load Test Data
# ==================================================================
test_ch0 = load_channel_data(test_probe, adc_num, 1)
test_ch1 = load_channel_data(test_probe, adc_num, 2)

# Stack test data
X_test = np.vstack([test_ch0, test_ch1])
y_test = np.concatenate([np.zeros(test_ch0.shape[0]), np.ones(test_ch1.shape[0])])

# Apply standardization using the loaded scaler
X_test_scaled = scaler.transform(X_test)

# Select only required 19 features from the 38 loaded dataset
X_test_selected = X_test_scaled[:, selected_feature_indices]

# ==================================================================
# Evaluate the Model
# ==================================================================
y_pred_selected = rf_selected.predict(X_test_selected)
y_probs_selected = rf_selected.predict_proba(X_test_selected)[:, 1]

# Print Model Performance
print("\nModel Performance Using 19 Features:")
print(f"Test Accuracy: {accuracy_score(y_test, y_pred_selected):.4f}")
print(f"Balanced Accuracy: {balanced_accuracy_score(y_test, y_pred_selected):.4f}")
print(f"MCC: {matthews_corrcoef(y_test, y_pred_selected):.4f}")
print(f"Log Loss: {log_loss(y_test, y_probs_selected):.4f}")
print(f"F1 Score: {f1_score(y_test, y_pred_selected):.4f}")
print(f"Recall: {recall_score(y_test, y_pred_selected):.4f}")
print(f"Precision: {precision_score(y_test, y_pred_selected):.4f}")
print(classification_report(y_test, y_pred_selected, target_names=['CH1', 'CH2']))

# Compute Confusion Matrix, FPR, and FNR
cm = confusion_matrix(y_test, y_pred_selected)
print("\nConfusion Matrix:")
print(cm)
fp_rate = cm[0][1] / (cm[0][1] + cm[0][0])
fn_rate = cm[1][0] / (cm[1][0] + cm[1][1])
print(f"False Positive Rate (FPR): {fp_rate:.4f}")
print(f"False Negative Rate (FNR): {fn_rate:.4f}")


✅ Loaded Probe3 ADC2 CH1: (498, 38)
✅ Loaded Probe3 ADC2 CH2: (498, 38)

Model Performance Using 19 Features:
Test Accuracy: 1.0000
Balanced Accuracy: 1.0000
MCC: 1.0000
Log Loss: 0.1617
F1 Score: 1.0000
Recall: 1.0000
Precision: 1.0000
              precision    recall  f1-score   support

         CH1       1.00      1.00      1.00       498
         CH2       1.00      1.00      1.00       498

    accuracy                           1.00       996
   macro avg       1.00      1.00      1.00       996
weighted avg       1.00      1.00      1.00       996


Confusion Matrix:
[[498   0]
 [  0 498]]
False Positive Rate (FPR): 0.0000
False Negative Rate (FNR): 0.0000


### ADC3 Probe3 Test at 200 MSPS

In [27]:
import numpy as np
import os
import joblib
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score, classification_report, confusion_matrix, log_loss,
    f1_score, recall_score, precision_score, matthews_corrcoef, balanced_accuracy_score
)

# ==================================================================
# Fixed Configuration
# ==================================================================
base_directory = "C:\\Users\\awm21\\Documents\\Probe_Vari\\Neural_Networks\\Features_Vitis"

adc_num = 3
test_probe_dataset = 3
sample_rate = 200

test_probe = f"Probe{test_probe_dataset}"

# Directory where the model and scaler were saved
model_dir = f"Models_test/19_Feature_Tuning_Probe1_to_Probe2_ADC1_200_MSPS"

# Load the trained model and scaler
model_path = os.path.join(model_dir, "RandomForest_Model.pkl")
scaler_path = os.path.join(model_dir, "Scaler.pkl")
features_path = os.path.join(model_dir, "Selected_Features.pkl")

if not os.path.exists(model_path) or not os.path.exists(scaler_path) or not os.path.exists(features_path):
    raise FileNotFoundError("Model, scaler, or selected features file not found.")

rf_selected = joblib.load(model_path)
scaler = joblib.load(scaler_path)
selected_feature_indices = joblib.load(features_path)

# List of all 38 loaded features
all_feature_names = [
    "Max_H", "Min_H", "Mean_H", "Std_H", "Mean Deviation_H", "RMS_H", "Skewness_H", "Kurtosis_H", "Peak-to-Peak_H", "Zero Crossing Rate_H",
    "Centroid_H", "Entropy_H", "Spread_H", "Skewness_Freq_H", "Mean_Freq_H", "Kurtosis_Freq_H", "Irregularity_H", "Variance_H", "Dominant_Freq_H",
    "Max_L", "Min_L", "Mean_L", "Std_L", "Mean Deviation_L", "RMS_L", "Skewness_L", "Kurtosis_L", "Peak-to-Peak_L", "Zero Crossing Rate_L",
    "Centroid_L", "Entropy_L", "Spread_L", "Skewness_Freq_L", "Mean_Freq_L", "Kurtosis_Freq_L", "Irregularity_L", "Variance_L"
]

# Get feature names for selected features
selected_features = [all_feature_names[i] for i in selected_feature_indices]

def load_channel_data(probe_name, adc_num, channel):
    """Load specific ADC/channel data for a probe and verify ADC consistency."""
    folder = os.path.join(base_directory, f"P{probe_name[-1]}_fd")
    filename = f'fd_{probe_name}_ADC{adc_num}_CH{channel}_{sample_rate}_MSPS.npy'
    file_path = os.path.join(folder, filename)
    
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"File not found: {file_path}")
    
    data = np.load(file_path)
    print(f"✅ Loaded {probe_name} ADC{adc_num} CH{channel}: {data.shape}")
    return data

# ==================================================================
# Load Test Data
# ==================================================================
test_ch0 = load_channel_data(test_probe, adc_num, 1)
test_ch1 = load_channel_data(test_probe, adc_num, 2)

# Stack test data
X_test = np.vstack([test_ch0, test_ch1])
y_test = np.concatenate([np.zeros(test_ch0.shape[0]), np.ones(test_ch1.shape[0])])

# Apply standardization using the loaded scaler
X_test_scaled = scaler.transform(X_test)

# Select only required 19 features from the 38 loaded dataset
X_test_selected = X_test_scaled[:, selected_feature_indices]

# ==================================================================
# Evaluate the Model
# ==================================================================
y_pred_selected = rf_selected.predict(X_test_selected)
y_probs_selected = rf_selected.predict_proba(X_test_selected)[:, 1]

# Print Model Performance
print("\nModel Performance Using 19 Features:")
print(f"Test Accuracy: {accuracy_score(y_test, y_pred_selected):.4f}")
print(f"Balanced Accuracy: {balanced_accuracy_score(y_test, y_pred_selected):.4f}")
print(f"MCC: {matthews_corrcoef(y_test, y_pred_selected):.4f}")
print(f"Log Loss: {log_loss(y_test, y_probs_selected):.4f}")
print(f"F1 Score: {f1_score(y_test, y_pred_selected):.4f}")
print(f"Recall: {recall_score(y_test, y_pred_selected):.4f}")
print(f"Precision: {precision_score(y_test, y_pred_selected):.4f}")
print(classification_report(y_test, y_pred_selected, target_names=['CH1', 'CH2']))

# Compute Confusion Matrix, FPR, and FNR
cm = confusion_matrix(y_test, y_pred_selected)
print("\nConfusion Matrix:")
print(cm)
fp_rate = cm[0][1] / (cm[0][1] + cm[0][0])
fn_rate = cm[1][0] / (cm[1][0] + cm[1][1])
print(f"False Positive Rate (FPR): {fp_rate:.4f}")
print(f"False Negative Rate (FNR): {fn_rate:.4f}")


✅ Loaded Probe3 ADC3 CH1: (498, 38)
✅ Loaded Probe3 ADC3 CH2: (498, 38)

Model Performance Using 19 Features:
Test Accuracy: 1.0000
Balanced Accuracy: 1.0000
MCC: 1.0000
Log Loss: 0.1577
F1 Score: 1.0000
Recall: 1.0000
Precision: 1.0000
              precision    recall  f1-score   support

         CH1       1.00      1.00      1.00       498
         CH2       1.00      1.00      1.00       498

    accuracy                           1.00       996
   macro avg       1.00      1.00      1.00       996
weighted avg       1.00      1.00      1.00       996


Confusion Matrix:
[[498   0]
 [  0 498]]
False Positive Rate (FPR): 0.0000
False Negative Rate (FNR): 0.0000


# 250 MSPS

## Training the model with 19 Features


In [28]:

import numpy as np
import os
import joblib
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import skew, kurtosis
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score, classification_report, confusion_matrix,
    roc_curve, auc, precision_recall_curve, matthews_corrcoef, 
    balanced_accuracy_score, log_loss, f1_score, recall_score, precision_score
)

# ==================================================================
# Fixed Configuration
# ==================================================================
base_directory = "C:\\Users\\awm21\\Documents\\Probe_Vari\\Neural_Networks\\Features_Vitis"

adc_num = 1
train_probe_dataset = 1
test_probe_dataset = 2
sample_rate = 250

train_probe = f"Probe{train_probe_dataset}"
test_probe = f"Probe{test_probe_dataset}"

# List of all 38 loaded features
all_feature_names = [
    "Max_H", "Min_H", "Mean_H", "Std_H", "Mean Deviation_H", "RMS_H", "Skewness_H", "Kurtosis_H", "Peak-to-Peak_H", "Zero Crossing Rate_H",
    "Centroid_H", "Entropy_H", "Spread_H", "Skewness_Freq_H", "Mean_Freq_H", "Kurtosis_Freq_H", "Irregularity_H", "Variance_H", "Dominant_Freq_H",
    "Max_L", "Min_L", "Mean_L", "Std_L", "Mean Deviation_L", "RMS_L", "Skewness_L", "Kurtosis_L", "Peak-to-Peak_L", "Zero Crossing Rate_L",
    "Centroid_L", "Entropy_L", "Spread_L", "Skewness_Freq_L", "Mean_Freq_L", "Kurtosis_Freq_L", "Irregularity_L", "Variance_L"
]

# Select only the 19 required features from the 38 loaded features
selected_features = [
    "Mean_H", "RMS_H", "Mean_L", "Variance_L", "RMS_L", "Mean_Freq_H", "Entropy_L", 
    "Max_H", "Entropy_H", "Mean Deviation_H", "Min_L", "Variance_H", "Mean_Freq_L", 
    "Min_H", "Mean Deviation_L", "Max_L", "Std_L", "Skewness_H", "Skewness_L"
]

# Get indices of selected features in the original dataset
selected_feature_indices = [all_feature_names.index(f) for f in selected_features]

def load_channel_data(probe_name, adc_num, channel):
    """Load specific ADC/channel data for a probe and verify ADC consistency."""
    folder = os.path.join(base_directory, f"P{probe_name[-1]}_fd")
    filename = f'fd_{probe_name}_ADC{adc_num}_CH{channel}_{sample_rate}_MSPS.npy'
    file_path = os.path.join(folder, filename)
    
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"File not found: {file_path}")
    
    data = np.load(file_path)
    print(f"✅ Loaded {probe_name} ADC{adc_num} CH{channel}: {data.shape}")
    return data

# ==================================================================
# Training & Test Data
# ==================================================================
train_ch0 = load_channel_data(train_probe, adc_num, 1)
train_ch1 = load_channel_data(train_probe, adc_num, 2)
test_ch0 = load_channel_data(test_probe, adc_num, 1)
test_ch1 = load_channel_data(test_probe, adc_num, 2)

# Stack training and test data
X_train = np.vstack([train_ch0, train_ch1])
y_train = np.concatenate([np.zeros(train_ch0.shape[0]), np.ones(train_ch1.shape[0])])
X_test = np.vstack([test_ch0, test_ch1])
y_test = np.concatenate([np.zeros(test_ch0.shape[0]), np.ones(test_ch1.shape[0])])

# ==================================================================
# Data Preprocessing
# ==================================================================
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Select only required 19 features from the 38 loaded dataset
X_train_selected = X_train_scaled[:, selected_feature_indices]
X_test_selected = X_test_scaled[:, selected_feature_indices]

# ==================================================================
# Train Model with Selected Features (19 Features)
# ==================================================================
rf = RandomForestClassifier(
    n_estimators=50,
    max_depth=10,
    min_samples_split=10,
    min_samples_leaf=5,
    max_features='sqrt',
    random_state=42
)
rf.fit(X_train_selected, y_train)

# Get feature importance only for selected features
feature_importances = rf.feature_importances_

# Print Selected Features and their Importances
print("\nSelected Features and Importances:")
for feature_idx, feature_name in zip(range(len(selected_features)), selected_features):
    print(f"{feature_name}: Importance {feature_importances[feature_idx]:.4f}")

# ==================================================================
# Retrain Model with Selected Features
# ==================================================================
rf_selected = RandomForestClassifier(
    n_estimators=50,
    max_depth=10,
    min_samples_split=10,
    min_samples_leaf=5,
    max_features='sqrt',
    random_state=42
)
rf_selected.fit(X_train_selected, y_train)

# Create a unique folder name dynamically
model_dir = f"Models_test/19_Feature_Tuning_{train_probe}_to_{test_probe}_ADC{adc_num}_{sample_rate}_MSPS"
os.makedirs(model_dir, exist_ok=True)

# Save the Model and Scaler
joblib.dump(rf_selected, os.path.join(model_dir, "RandomForest_Model.pkl"))
joblib.dump(scaler, os.path.join(model_dir, "Scaler.pkl"))
joblib.dump(selected_feature_indices, os.path.join(model_dir, "Selected_Features.pkl"))

# Predictions
y_pred_selected = rf_selected.predict(X_test_selected)
y_probs_selected = rf_selected.predict_proba(X_test_selected)[:, 1]

# ==================================================================
# Performance Metrics After Feature Selection
# ==================================================================
print("\nModel Performance Using 19 Features:")
print(f"Test Accuracy: {accuracy_score(y_test, y_pred_selected):.4f}")
print(f"Balanced Accuracy: {balanced_accuracy_score(y_test, y_pred_selected):.4f}")
print(f"MCC: {matthews_corrcoef(y_test, y_pred_selected):.4f}")
print(f"Log Loss: {log_loss(y_test, y_probs_selected):.4f}")
print(f"F1 Score: {f1_score(y_test, y_pred_selected):.4f}")
print(f"Recall: {recall_score(y_test, y_pred_selected):.4f}")
print(f"Precision: {precision_score(y_test, y_pred_selected):.4f}")
print(classification_report(y_test, y_pred_selected, target_names=['CH1', 'CH2']))

# Compute Confusion Matrix, FPR, and FNR
cm = confusion_matrix(y_test, y_pred_selected)
print("\nConfusion Matrix:")
print(cm)
fp_rate = cm[0][1] / (cm[0][1] + cm[0][0])
fn_rate = cm[1][0] / (cm[1][0] + cm[1][1])
print(f"False Positive Rate (FPR): {fp_rate:.4f}")
print(f"False Negative Rate (FNR): {fn_rate:.4f}")


✅ Loaded Probe1 ADC1 CH1: (398, 38)
✅ Loaded Probe1 ADC1 CH2: (398, 38)
✅ Loaded Probe2 ADC1 CH1: (398, 38)
✅ Loaded Probe2 ADC1 CH2: (398, 38)

Selected Features and Importances:
Mean_H: Importance 0.1200
RMS_H: Importance 0.1202
Mean_L: Importance 0.1000
Variance_L: Importance 0.0800
RMS_L: Importance 0.0601
Mean_Freq_H: Importance 0.0000
Entropy_L: Importance 0.0398
Max_H: Importance 0.0800
Entropy_H: Importance 0.0000
Mean Deviation_H: Importance 0.1000
Min_L: Importance 0.0799
Variance_H: Importance 0.1000
Mean_Freq_L: Importance 0.0000
Min_H: Importance 0.0000
Mean Deviation_L: Importance 0.1200
Max_L: Importance 0.0000
Std_L: Importance 0.0000
Skewness_H: Importance 0.0000
Skewness_L: Importance 0.0000

Model Performance Using 19 Features:
Test Accuracy: 1.0000
Balanced Accuracy: 1.0000
MCC: 1.0000
Log Loss: 0.0013
F1 Score: 1.0000
Recall: 1.0000
Precision: 1.0000
              precision    recall  f1-score   support

         CH1       1.00      1.00      1.00       398
       

## Testing on the models trained from above


### ADC2 Probe3 Test at 250 MSPS


In [29]:

import numpy as np
import os
import joblib
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score, classification_report, confusion_matrix, log_loss,
    f1_score, recall_score, precision_score, matthews_corrcoef, balanced_accuracy_score
)

# ==================================================================
# Fixed Configuration
# ==================================================================
base_directory = "C:\\Users\\awm21\\Documents\\Probe_Vari\\Neural_Networks\\Features_Vitis"

adc_num = 2
test_probe_dataset = 3
sample_rate = 250

test_probe = f"Probe{test_probe_dataset}"

# Directory where the model and scaler were saved
model_dir = f"Models_test/19_Feature_Tuning_Probe1_to_Probe2_ADC1_250_MSPS"

# Load the trained model and scaler
model_path = os.path.join(model_dir, "RandomForest_Model.pkl")
scaler_path = os.path.join(model_dir, "Scaler.pkl")
features_path = os.path.join(model_dir, "Selected_Features.pkl")

if not os.path.exists(model_path) or not os.path.exists(scaler_path) or not os.path.exists(features_path):
    raise FileNotFoundError("Model, scaler, or selected features file not found.")

rf_selected = joblib.load(model_path)
scaler = joblib.load(scaler_path)
selected_feature_indices = joblib.load(features_path)

# List of all 38 loaded features
all_feature_names = [
    "Max_H", "Min_H", "Mean_H", "Std_H", "Mean Deviation_H", "RMS_H", "Skewness_H", "Kurtosis_H", "Peak-to-Peak_H", "Zero Crossing Rate_H",
    "Centroid_H", "Entropy_H", "Spread_H", "Skewness_Freq_H", "Mean_Freq_H", "Kurtosis_Freq_H", "Irregularity_H", "Variance_H", "Dominant_Freq_H",
    "Max_L", "Min_L", "Mean_L", "Std_L", "Mean Deviation_L", "RMS_L", "Skewness_L", "Kurtosis_L", "Peak-to-Peak_L", "Zero Crossing Rate_L",
    "Centroid_L", "Entropy_L", "Spread_L", "Skewness_Freq_L", "Mean_Freq_L", "Kurtosis_Freq_L", "Irregularity_L", "Variance_L"
]

# Get feature names for selected features
selected_features = [all_feature_names[i] for i in selected_feature_indices]

def load_channel_data(probe_name, adc_num, channel):
    """Load specific ADC/channel data for a probe and verify ADC consistency."""
    folder = os.path.join(base_directory, f"P{probe_name[-1]}_fd")
    filename = f'fd_{probe_name}_ADC{adc_num}_CH{channel}_{sample_rate}_MSPS.npy'
    file_path = os.path.join(folder, filename)
    
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"File not found: {file_path}")
    
    data = np.load(file_path)
    print(f"✅ Loaded {probe_name} ADC{adc_num} CH{channel}: {data.shape}")
    return data

# ==================================================================
# Load Test Data
# ==================================================================
test_ch0 = load_channel_data(test_probe, adc_num, 1)
test_ch1 = load_channel_data(test_probe, adc_num, 2)

# Stack test data
X_test = np.vstack([test_ch0, test_ch1])
y_test = np.concatenate([np.zeros(test_ch0.shape[0]), np.ones(test_ch1.shape[0])])

# Apply standardization using the loaded scaler
X_test_scaled = scaler.transform(X_test)

# Select only required 19 features from the 38 loaded dataset
X_test_selected = X_test_scaled[:, selected_feature_indices]

# ==================================================================
# Evaluate the Model
# ==================================================================
y_pred_selected = rf_selected.predict(X_test_selected)
y_probs_selected = rf_selected.predict_proba(X_test_selected)[:, 1]

# Print Model Performance
print("\nModel Performance Using 19 Features:")
print(f"Test Accuracy: {accuracy_score(y_test, y_pred_selected):.4f}")
print(f"Balanced Accuracy: {balanced_accuracy_score(y_test, y_pred_selected):.4f}")
print(f"MCC: {matthews_corrcoef(y_test, y_pred_selected):.4f}")
print(f"Log Loss: {log_loss(y_test, y_probs_selected):.4f}")
print(f"F1 Score: {f1_score(y_test, y_pred_selected):.4f}")
print(f"Recall: {recall_score(y_test, y_pred_selected):.4f}")
print(f"Precision: {precision_score(y_test, y_pred_selected):.4f}")
print(classification_report(y_test, y_pred_selected, target_names=['CH1', 'CH2']))

# Compute Confusion Matrix, FPR, and FNR
cm = confusion_matrix(y_test, y_pred_selected)
print("\nConfusion Matrix:")
print(cm)
fp_rate = cm[0][1] / (cm[0][1] + cm[0][0])
fn_rate = cm[1][0] / (cm[1][0] + cm[1][1])
print(f"False Positive Rate (FPR): {fp_rate:.4f}")
print(f"False Negative Rate (FNR): {fn_rate:.4f}")


✅ Loaded Probe3 ADC2 CH1: (398, 38)
✅ Loaded Probe3 ADC2 CH2: (398, 38)

Model Performance Using 19 Features:
Test Accuracy: 1.0000
Balanced Accuracy: 1.0000
MCC: 1.0000
Log Loss: 0.0289
F1 Score: 1.0000
Recall: 1.0000
Precision: 1.0000
              precision    recall  f1-score   support

         CH1       1.00      1.00      1.00       398
         CH2       1.00      1.00      1.00       398

    accuracy                           1.00       796
   macro avg       1.00      1.00      1.00       796
weighted avg       1.00      1.00      1.00       796


Confusion Matrix:
[[398   0]
 [  0 398]]
False Positive Rate (FPR): 0.0000
False Negative Rate (FNR): 0.0000


### ADC3 Probe3 Test at 250MSPS

In [30]:

import numpy as np
import os
import joblib
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score, classification_report, confusion_matrix, log_loss,
    f1_score, recall_score, precision_score, matthews_corrcoef, balanced_accuracy_score
)

# ==================================================================
# Fixed Configuration
# ==================================================================
base_directory = "C:\\Users\\awm21\\Documents\\Probe_Vari\\Neural_Networks\\Features_Vitis"

adc_num = 2
test_probe_dataset = 3
sample_rate = 250

test_probe = f"Probe{test_probe_dataset}"

# Directory where the model and scaler were saved
model_dir = f"Models_test/19_Feature_Tuning_Probe1_to_Probe2_ADC1_250_MSPS"

# Load the trained model and scaler
model_path = os.path.join(model_dir, "RandomForest_Model.pkl")
scaler_path = os.path.join(model_dir, "Scaler.pkl")
features_path = os.path.join(model_dir, "Selected_Features.pkl")

if not os.path.exists(model_path) or not os.path.exists(scaler_path) or not os.path.exists(features_path):
    raise FileNotFoundError("Model, scaler, or selected features file not found.")

rf_selected = joblib.load(model_path)
scaler = joblib.load(scaler_path)
selected_feature_indices = joblib.load(features_path)

# List of all 38 loaded features
all_feature_names = [
    "Max_H", "Min_H", "Mean_H", "Std_H", "Mean Deviation_H", "RMS_H", "Skewness_H", "Kurtosis_H", "Peak-to-Peak_H", "Zero Crossing Rate_H",
    "Centroid_H", "Entropy_H", "Spread_H", "Skewness_Freq_H", "Mean_Freq_H", "Kurtosis_Freq_H", "Irregularity_H", "Variance_H", "Dominant_Freq_H",
    "Max_L", "Min_L", "Mean_L", "Std_L", "Mean Deviation_L", "RMS_L", "Skewness_L", "Kurtosis_L", "Peak-to-Peak_L", "Zero Crossing Rate_L",
    "Centroid_L", "Entropy_L", "Spread_L", "Skewness_Freq_L", "Mean_Freq_L", "Kurtosis_Freq_L", "Irregularity_L", "Variance_L"
]

# Get feature names for selected features
selected_features = [all_feature_names[i] for i in selected_feature_indices]

def load_channel_data(probe_name, adc_num, channel):
    """Load specific ADC/channel data for a probe and verify ADC consistency."""
    folder = os.path.join(base_directory, f"P{probe_name[-1]}_fd")
    filename = f'fd_{probe_name}_ADC{adc_num}_CH{channel}_{sample_rate}_MSPS.npy'
    file_path = os.path.join(folder, filename)
    
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"File not found: {file_path}")
    
    data = np.load(file_path)
    print(f"✅ Loaded {probe_name} ADC{adc_num} CH{channel}: {data.shape}")
    return data

# ==================================================================
# Load Test Data
# ==================================================================
test_ch0 = load_channel_data(test_probe, adc_num, 1)
test_ch1 = load_channel_data(test_probe, adc_num, 2)

# Stack test data
X_test = np.vstack([test_ch0, test_ch1])
y_test = np.concatenate([np.zeros(test_ch0.shape[0]), np.ones(test_ch1.shape[0])])

# Apply standardization using the loaded scaler
X_test_scaled = scaler.transform(X_test)

# Select only required 19 features from the 38 loaded dataset
X_test_selected = X_test_scaled[:, selected_feature_indices]

# ==================================================================
# Evaluate the Model
# ==================================================================
y_pred_selected = rf_selected.predict(X_test_selected)
y_probs_selected = rf_selected.predict_proba(X_test_selected)[:, 1]

# Print Model Performance
print("\nModel Performance Using 19 Features:")
print(f"Test Accuracy: {accuracy_score(y_test, y_pred_selected):.4f}")
print(f"Balanced Accuracy: {balanced_accuracy_score(y_test, y_pred_selected):.4f}")
print(f"MCC: {matthews_corrcoef(y_test, y_pred_selected):.4f}")
print(f"Log Loss: {log_loss(y_test, y_probs_selected):.4f}")
print(f"F1 Score: {f1_score(y_test, y_pred_selected):.4f}")
print(f"Recall: {recall_score(y_test, y_pred_selected):.4f}")
print(f"Precision: {precision_score(y_test, y_pred_selected):.4f}")
print(classification_report(y_test, y_pred_selected, target_names=['CH1', 'CH2']))

# Compute Confusion Matrix, FPR, and FNR
cm = confusion_matrix(y_test, y_pred_selected)
print("\nConfusion Matrix:")
print(cm)
fp_rate = cm[0][1] / (cm[0][1] + cm[0][0])
fn_rate = cm[1][0] / (cm[1][0] + cm[1][1])
print(f"False Positive Rate (FPR): {fp_rate:.4f}")
print(f"False Negative Rate (FNR): {fn_rate:.4f}")


✅ Loaded Probe3 ADC2 CH1: (398, 38)
✅ Loaded Probe3 ADC2 CH2: (398, 38)

Model Performance Using 19 Features:
Test Accuracy: 1.0000
Balanced Accuracy: 1.0000
MCC: 1.0000
Log Loss: 0.0289
F1 Score: 1.0000
Recall: 1.0000
Precision: 1.0000
              precision    recall  f1-score   support

         CH1       1.00      1.00      1.00       398
         CH2       1.00      1.00      1.00       398

    accuracy                           1.00       796
   macro avg       1.00      1.00      1.00       796
weighted avg       1.00      1.00      1.00       796


Confusion Matrix:
[[398   0]
 [  0 398]]
False Positive Rate (FPR): 0.0000
False Negative Rate (FNR): 0.0000


# 300 MSPS

## Training the model with 19 Features


In [31]:

import numpy as np
import os
import joblib
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import skew, kurtosis
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score, classification_report, confusion_matrix,
    roc_curve, auc, precision_recall_curve, matthews_corrcoef, 
    balanced_accuracy_score, log_loss, f1_score, recall_score, precision_score
)

# ==================================================================
# Fixed Configuration
# ==================================================================
base_directory = "C:\\Users\\awm21\\Documents\\Probe_Vari\\Neural_Networks\\Features_Vitis"

adc_num = 1
train_probe_dataset = 1
test_probe_dataset = 2
sample_rate = 300

train_probe = f"Probe{train_probe_dataset}"
test_probe = f"Probe{test_probe_dataset}"

# List of all 38 loaded features
all_feature_names = [
    "Max_H", "Min_H", "Mean_H", "Std_H", "Mean Deviation_H", "RMS_H", "Skewness_H", "Kurtosis_H", "Peak-to-Peak_H", "Zero Crossing Rate_H",
    "Centroid_H", "Entropy_H", "Spread_H", "Skewness_Freq_H", "Mean_Freq_H", "Kurtosis_Freq_H", "Irregularity_H", "Variance_H", "Dominant_Freq_H",
    "Max_L", "Min_L", "Mean_L", "Std_L", "Mean Deviation_L", "RMS_L", "Skewness_L", "Kurtosis_L", "Peak-to-Peak_L", "Zero Crossing Rate_L",
    "Centroid_L", "Entropy_L", "Spread_L", "Skewness_Freq_L", "Mean_Freq_L", "Kurtosis_Freq_L", "Irregularity_L", "Variance_L"
]

# Select only the 19 required features from the 38 loaded features
selected_features = [
    "Mean_H", "RMS_H", "Mean_L", "Variance_L", "RMS_L", "Mean_Freq_H", "Entropy_L", 
    "Max_H", "Entropy_H", "Mean Deviation_H", "Min_L", "Variance_H", "Mean_Freq_L", 
    "Min_H", "Mean Deviation_L", "Max_L", "Std_L", "Skewness_H", "Skewness_L"
]

# Get indices of selected features in the original dataset
selected_feature_indices = [all_feature_names.index(f) for f in selected_features]

def load_channel_data(probe_name, adc_num, channel):
    """Load specific ADC/channel data for a probe and verify ADC consistency."""
    folder = os.path.join(base_directory, f"P{probe_name[-1]}_fd")
    filename = f'fd_{probe_name}_ADC{adc_num}_CH{channel}_{sample_rate}_MSPS.npy'
    file_path = os.path.join(folder, filename)
    
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"File not found: {file_path}")
    
    data = np.load(file_path)
    print(f"✅ Loaded {probe_name} ADC{adc_num} CH{channel}: {data.shape}")
    return data

# ==================================================================
# Training & Test Data
# ==================================================================
train_ch0 = load_channel_data(train_probe, adc_num, 1)
train_ch1 = load_channel_data(train_probe, adc_num, 2)
test_ch0 = load_channel_data(test_probe, adc_num, 1)
test_ch1 = load_channel_data(test_probe, adc_num, 2)

# Stack training and test data
X_train = np.vstack([train_ch0, train_ch1])
y_train = np.concatenate([np.zeros(train_ch0.shape[0]), np.ones(train_ch1.shape[0])])
X_test = np.vstack([test_ch0, test_ch1])
y_test = np.concatenate([np.zeros(test_ch0.shape[0]), np.ones(test_ch1.shape[0])])

# ==================================================================
# Data Preprocessing
# ==================================================================
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Select only required 19 features from the 38 loaded dataset
X_train_selected = X_train_scaled[:, selected_feature_indices]
X_test_selected = X_test_scaled[:, selected_feature_indices]

# ==================================================================
# Train Model with Selected Features (19 Features)
# ==================================================================
rf = RandomForestClassifier(
    n_estimators=50,
    max_depth=10,
    min_samples_split=10,
    min_samples_leaf=5,
    max_features='sqrt',
    random_state=42
)
rf.fit(X_train_selected, y_train)

# Get feature importance only for selected features
feature_importances = rf.feature_importances_

# Print Selected Features and their Importances
print("\nSelected Features and Importances:")
for feature_idx, feature_name in zip(range(len(selected_features)), selected_features):
    print(f"{feature_name}: Importance {feature_importances[feature_idx]:.4f}")

# ==================================================================
# Retrain Model with Selected Features
# ==================================================================
rf_selected = RandomForestClassifier(
    n_estimators=50,
    max_depth=10,
    min_samples_split=10,
    min_samples_leaf=5,
    max_features='sqrt',
    random_state=42
)
rf_selected.fit(X_train_selected, y_train)

# Create a unique folder name dynamically
model_dir = f"Models_test/19_Feature_Tuning_{train_probe}_to_{test_probe}_ADC{adc_num}_{sample_rate}_MSPS"
os.makedirs(model_dir, exist_ok=True)

# Save the Model and Scaler
joblib.dump(rf_selected, os.path.join(model_dir, "RandomForest_Model.pkl"))
joblib.dump(scaler, os.path.join(model_dir, "Scaler.pkl"))
joblib.dump(selected_feature_indices, os.path.join(model_dir, "Selected_Features.pkl"))

# Predictions
y_pred_selected = rf_selected.predict(X_test_selected)
y_probs_selected = rf_selected.predict_proba(X_test_selected)[:, 1]

# ==================================================================
# Performance Metrics After Feature Selection
# ==================================================================
print("\nModel Performance Using 19 Features:")
print(f"Test Accuracy: {accuracy_score(y_test, y_pred_selected):.4f}")
print(f"Balanced Accuracy: {balanced_accuracy_score(y_test, y_pred_selected):.4f}")
print(f"MCC: {matthews_corrcoef(y_test, y_pred_selected):.4f}")
print(f"Log Loss: {log_loss(y_test, y_probs_selected):.4f}")
print(f"F1 Score: {f1_score(y_test, y_pred_selected):.4f}")
print(f"Recall: {recall_score(y_test, y_pred_selected):.4f}")
print(f"Precision: {precision_score(y_test, y_pred_selected):.4f}")
print(classification_report(y_test, y_pred_selected, target_names=['CH1', 'CH2']))

# Compute Confusion Matrix, FPR, and FNR
cm = confusion_matrix(y_test, y_pred_selected)
print("\nConfusion Matrix:")
print(cm)
fp_rate = cm[0][1] / (cm[0][1] + cm[0][0])
fn_rate = cm[1][0] / (cm[1][0] + cm[1][1])
print(f"False Positive Rate (FPR): {fp_rate:.4f}")
print(f"False Negative Rate (FNR): {fn_rate:.4f}")


✅ Loaded Probe1 ADC1 CH1: (332, 38)
✅ Loaded Probe1 ADC1 CH2: (332, 38)
✅ Loaded Probe2 ADC1 CH1: (332, 38)
✅ Loaded Probe2 ADC1 CH2: (332, 38)

Selected Features and Importances:
Mean_H: Importance 0.1000
RMS_H: Importance 0.0800
Mean_L: Importance 0.0400
Variance_L: Importance 0.0800
RMS_L: Importance 0.0000
Mean_Freq_H: Importance 0.1000
Entropy_L: Importance 0.1000
Max_H: Importance 0.0800
Entropy_H: Importance 0.0600
Mean Deviation_H: Importance 0.0600
Min_L: Importance 0.0400
Variance_H: Importance 0.0600
Mean_Freq_L: Importance 0.0800
Min_H: Importance 0.0000
Mean Deviation_L: Importance 0.0800
Max_L: Importance 0.0000
Std_L: Importance 0.0400
Skewness_H: Importance 0.0000
Skewness_L: Importance 0.0000

Model Performance Using 19 Features:
Test Accuracy: 1.0000
Balanced Accuracy: 1.0000
MCC: 1.0000
Log Loss: 0.0022
F1 Score: 1.0000
Recall: 1.0000
Precision: 1.0000
              precision    recall  f1-score   support

         CH1       1.00      1.00      1.00       332
       

## Testing on the models trained from above


### ADC2 Probe3 Test at 300MSPS

In [43]:

import numpy as np
import os
import joblib
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score, classification_report, confusion_matrix, log_loss,
    f1_score, recall_score, precision_score, matthews_corrcoef, balanced_accuracy_score
)

# ==================================================================
# Fixed Configuration
# ==================================================================
base_directory = "C:\\Users\\awm21\\Documents\\Probe_Vari\\Neural_Networks\\Features_Vitis"

adc_num = 2
test_probe_dataset = 3
sample_rate = 300

test_probe = f"Probe{test_probe_dataset}"

# Directory where the model and scaler were saved
model_dir = f"Models_test/19_Feature_Tuning_Probe1_to_Probe2_ADC1_300_MSPS"

# Load the trained model and scaler
model_path = os.path.join(model_dir, "RandomForest_Model.pkl")
scaler_path = os.path.join(model_dir, "Scaler.pkl")
features_path = os.path.join(model_dir, "Selected_Features.pkl")

if not os.path.exists(model_path) or not os.path.exists(scaler_path) or not os.path.exists(features_path):
    raise FileNotFoundError("Model, scaler, or selected features file not found.")

rf_selected = joblib.load(model_path)
scaler = joblib.load(scaler_path)
selected_feature_indices = joblib.load(features_path)

# List of all 38 loaded features
all_feature_names = [
    "Max_H", "Min_H", "Mean_H", "Std_H", "Mean Deviation_H", "RMS_H", "Skewness_H", "Kurtosis_H", "Peak-to-Peak_H", "Zero Crossing Rate_H",
    "Centroid_H", "Entropy_H", "Spread_H", "Skewness_Freq_H", "Mean_Freq_H", "Kurtosis_Freq_H", "Irregularity_H", "Variance_H", "Dominant_Freq_H",
    "Max_L", "Min_L", "Mean_L", "Std_L", "Mean Deviation_L", "RMS_L", "Skewness_L", "Kurtosis_L", "Peak-to-Peak_L", "Zero Crossing Rate_L",
    "Centroid_L", "Entropy_L", "Spread_L", "Skewness_Freq_L", "Mean_Freq_L", "Kurtosis_Freq_L", "Irregularity_L", "Variance_L"
]

# Get feature names for selected features
selected_features = [all_feature_names[i] for i in selected_feature_indices]

def load_channel_data(probe_name, adc_num, channel):
    """Load specific ADC/channel data for a probe and verify ADC consistency."""
    folder = os.path.join(base_directory, f"P{probe_name[-1]}_fd")
    filename = f'fd_{probe_name}_ADC{adc_num}_CH{channel}_{sample_rate}_MSPS.npy'
    file_path = os.path.join(folder, filename)
    
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"File not found: {file_path}")
    
    data = np.load(file_path)
    print(f"✅ Loaded {probe_name} ADC{adc_num} CH{channel}: {data.shape}")
    return data

# ==================================================================
# Load Test Data
# ==================================================================
test_ch0 = load_channel_data(test_probe, adc_num, 1)
test_ch1 = load_channel_data(test_probe, adc_num, 2)

# Stack test data
X_test = np.vstack([test_ch0, test_ch1])
y_test = np.concatenate([np.zeros(test_ch0.shape[0]), np.ones(test_ch1.shape[0])])

# Apply standardization using the loaded scaler
X_test_scaled = scaler.transform(X_test)

# Select only required 19 features from the 38 loaded dataset
X_test_selected = X_test_scaled[:, selected_feature_indices]

# ==================================================================
# Evaluate the Model
# ==================================================================
y_pred_selected = rf_selected.predict(X_test_selected)
y_probs_selected = rf_selected.predict_proba(X_test_selected)[:, 1]

# Print Model Performance
print("\nModel Performance Using 19 Features:")
print(f"Test Accuracy: {accuracy_score(y_test, y_pred_selected):.4f}")
print(f"Balanced Accuracy: {balanced_accuracy_score(y_test, y_pred_selected):.4f}")
print(f"MCC: {matthews_corrcoef(y_test, y_pred_selected):.4f}")
print(f"Log Loss: {log_loss(y_test, y_probs_selected):.4f}")
print(f"F1 Score: {f1_score(y_test, y_pred_selected):.4f}")
print(f"Recall: {recall_score(y_test, y_pred_selected):.4f}")
print(f"Precision: {precision_score(y_test, y_pred_selected):.4f}")
print(classification_report(y_test, y_pred_selected, target_names=['CH1', 'CH2']))

# Compute Confusion Matrix, FPR, and FNR
cm = confusion_matrix(y_test, y_pred_selected)
print("\nConfusion Matrix:")
print(cm)
fp_rate = cm[0][1] / (cm[0][1] + cm[0][0])
fn_rate = cm[1][0] / (cm[1][0] + cm[1][1])
print(f"False Positive Rate (FPR): {fp_rate:.4f}")
print(f"False Negative Rate (FNR): {fn_rate:.4f}")


✅ Loaded Probe3 ADC2 CH1: (331, 38)
✅ Loaded Probe3 ADC2 CH2: (332, 38)

Model Performance Using 19 Features:
Test Accuracy: 1.0000
Balanced Accuracy: 1.0000
MCC: 1.0000
Log Loss: 0.0616
F1 Score: 1.0000
Recall: 1.0000
Precision: 1.0000
              precision    recall  f1-score   support

         CH1       1.00      1.00      1.00       331
         CH2       1.00      1.00      1.00       332

    accuracy                           1.00       663
   macro avg       1.00      1.00      1.00       663
weighted avg       1.00      1.00      1.00       663


Confusion Matrix:
[[331   0]
 [  0 332]]
False Positive Rate (FPR): 0.0000
False Negative Rate (FNR): 0.0000


### ADC3 Probe3 Test at 300 MSPS

In [45]:

import numpy as np
import os
import joblib
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score, classification_report, confusion_matrix, log_loss,
    f1_score, recall_score, precision_score, matthews_corrcoef, balanced_accuracy_score
)

# ==================================================================
# Fixed Configuration
# ==================================================================
base_directory = "C:\\Users\\awm21\\Documents\\Probe_Vari\\Neural_Networks\\Features_Vitis"

adc_num = 3
test_probe_dataset = 3
sample_rate = 300

test_probe = f"Probe{test_probe_dataset}"

# Directory where the model and scaler were saved
model_dir = f"Models_test/19_Feature_Tuning_Probe1_to_Probe2_ADC1_300_MSPS"

# Load the trained model and scaler
model_path = os.path.join(model_dir, "RandomForest_Model.pkl")
scaler_path = os.path.join(model_dir, "Scaler.pkl")
features_path = os.path.join(model_dir, "Selected_Features.pkl")

if not os.path.exists(model_path) or not os.path.exists(scaler_path) or not os.path.exists(features_path):
    raise FileNotFoundError("Model, scaler, or selected features file not found.")

rf_selected = joblib.load(model_path)
scaler = joblib.load(scaler_path)
selected_feature_indices = joblib.load(features_path)

# List of all 38 loaded features
all_feature_names = [
    "Max_H", "Min_H", "Mean_H", "Std_H", "Mean Deviation_H", "RMS_H", "Skewness_H", "Kurtosis_H", "Peak-to-Peak_H", "Zero Crossing Rate_H",
    "Centroid_H", "Entropy_H", "Spread_H", "Skewness_Freq_H", "Mean_Freq_H", "Kurtosis_Freq_H", "Irregularity_H", "Variance_H", "Dominant_Freq_H",
    "Max_L", "Min_L", "Mean_L", "Std_L", "Mean Deviation_L", "RMS_L", "Skewness_L", "Kurtosis_L", "Peak-to-Peak_L", "Zero Crossing Rate_L",
    "Centroid_L", "Entropy_L", "Spread_L", "Skewness_Freq_L", "Mean_Freq_L", "Kurtosis_Freq_L", "Irregularity_L", "Variance_L"
]

# Get feature names for selected features
selected_features = [all_feature_names[i] for i in selected_feature_indices]

def load_channel_data(probe_name, adc_num, channel):
    """Load specific ADC/channel data for a probe and verify ADC consistency."""
    folder = os.path.join(base_directory, f"P{probe_name[-1]}_fd")
    filename = f'fd_{probe_name}_ADC{adc_num}_CH{channel}_{sample_rate}_MSPS.npy'
    file_path = os.path.join(folder, filename)
    
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"File not found: {file_path}")
    
    data = np.load(file_path)
    print(f"✅ Loaded {probe_name} ADC{adc_num} CH{channel}: {data.shape}")
    return data

# ==================================================================
# Load Test Data
# ==================================================================
test_ch0 = load_channel_data(test_probe, adc_num, 1)
test_ch1 = load_channel_data(test_probe, adc_num, 2)

# Stack test data
X_test = np.vstack([test_ch0, test_ch1])
y_test = np.concatenate([np.zeros(test_ch0.shape[0]), np.ones(test_ch1.shape[0])])

# Apply standardization using the loaded scaler
X_test_scaled = scaler.transform(X_test)

# Select only required 19 features from the 38 loaded dataset
X_test_selected = X_test_scaled[:, selected_feature_indices]

# ==================================================================
# Evaluate the Model
# ==================================================================
y_pred_selected = rf_selected.predict(X_test_selected)
y_probs_selected = rf_selected.predict_proba(X_test_selected)[:, 1]

# Print Model Performance
print("\nModel Performance Using 19 Features:")
print(f"Test Accuracy: {accuracy_score(y_test, y_pred_selected):.4f}")
print(f"Balanced Accuracy: {balanced_accuracy_score(y_test, y_pred_selected):.4f}")
print(f"MCC: {matthews_corrcoef(y_test, y_pred_selected):.4f}")
print(f"Log Loss: {log_loss(y_test, y_probs_selected):.4f}")
print(f"F1 Score: {f1_score(y_test, y_pred_selected):.4f}")
print(f"Recall: {recall_score(y_test, y_pred_selected):.4f}")
print(f"Precision: {precision_score(y_test, y_pred_selected):.4f}")
print(classification_report(y_test, y_pred_selected, target_names=['CH1', 'CH2']))

# Compute Confusion Matrix, FPR, and FNR
cm = confusion_matrix(y_test, y_pred_selected)
print("\nConfusion Matrix:")
print(cm)
fp_rate = cm[0][1] / (cm[0][1] + cm[0][0])
fn_rate = cm[1][0] / (cm[1][0] + cm[1][1])
print(f"False Positive Rate (FPR): {fp_rate:.4f}")
print(f"False Negative Rate (FNR): {fn_rate:.4f}")


✅ Loaded Probe3 ADC3 CH1: (331, 38)
✅ Loaded Probe3 ADC3 CH2: (332, 38)

Model Performance Using 19 Features:
Test Accuracy: 0.9955
Balanced Accuracy: 0.9955
MCC: 0.9910
Log Loss: 0.1455
F1 Score: 0.9955
Recall: 0.9910
Precision: 1.0000
              precision    recall  f1-score   support

         CH1       0.99      1.00      1.00       331
         CH2       1.00      0.99      1.00       332

    accuracy                           1.00       663
   macro avg       1.00      1.00      1.00       663
weighted avg       1.00      1.00      1.00       663


Confusion Matrix:
[[331   0]
 [  3 329]]
False Positive Rate (FPR): 0.0000
False Negative Rate (FNR): 0.0090
